In [1]:
#importing packages
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import time
import torchvision.transforms as transforms

In [2]:
#loading data
with open('data_full.json') as file:
    oos = json.load(file)

In [3]:
#listing files
oos.keys()

dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])

In [4]:
#assembling files into pandas dataframes
temp = oos['oos_val']
oos_val = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})
temp = oos['val']
val = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})
temp = oos['train']
train = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})
temp = oos['oos_test']
oos_test = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})
temp = oos['test']
test = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})
temp = oos['oos_train']
oos_train = pd.DataFrame(temp).rename(columns={0:"query", 1:"domain"})

In [5]:
#inspecting dataframes
print(oos_val.head())
print(val.head())
print(train.head())
print(oos_test.head())
print(test.head())
print(oos_train.head())


                                               query domain
0  set a warning for when my bank account starts ...    oos
1                                 a show on broadway    oos
2                 who has the best record in the nfl    oos
3                 how do i find the area of a circle    oos
4                  how many onions do i have on hand    oos
                                       query     domain
0   in spanish, meet me tomorrow is said how  translate
1     in french, how do i say, see you later  translate
2           how do you say hello in japanese  translate
3  how do i ask about the weather in chinese  translate
4  how can i say "cancel my order" in french  translate
                                               query     domain
0  what expression would i use to say i love you ...  translate
1  can you tell me how to say 'i do not speak muc...  translate
2  what is the equivalent of, 'life is good' in f...  translate
3  tell me how to say, 'it is a beautiful mornin

In [6]:
#our training protocol will use the 'out of scope' training data to train for this class, so appending the data to one dataframe
train = train.append(oos_train, ignore_index=True)

In [7]:
#defining the vectorizer that will be used for this dataset
vectorizer = TfidfVectorizer()

In [8]:
#fitting the TFIDF vectorizer to the training data queries and transforming it
X = vectorizer.fit_transform(train['query']).toarray()

In [9]:
# checking the size of the array
X.shape

(15100, 5146)

In [11]:
#inspecting vectorizer features.
print(vectorizer.get_feature_names()[:20])
print(vectorizer.get_feature_names()[-20:])

['00', '000', '005', '00am', '00pm', '01', '02', '03', '05', '098098', '10', '100', '1000', '10000', '100000', '10294', '104', '10500', '10am', '10kg']
['zales', 'zander', 'zazie', 'zealand', 'zebras', 'zen', 'zenith', 'zepher', 'zephers', 'zeppelin', 'zesty', 'zeus', 'zion', 'zippy', 'zippys', 'ziti', 'zombie', 'zone', 'zoo', 'zulu']


These aren't all words, but this reflects the fact that inputs will not always be words. Equally some are variations (zippy/zippys) but these will be preserved to minimise the work done in preprocessing input data during deployment.

In [12]:
#making dataframe of X
X_df = pd.DataFrame(X)

In [13]:
#shape of X dataframe
X_df.shape

(15100, 5146)

In [14]:
#shape of initial training dataframe
train.shape

(15100, 2)

In [15]:
#joining training dataframe to vectorized words
train_vec = train.join(X_df)

In [16]:
#finding shape of joined dataframe
train_vec.shape

(15100, 5148)

In [17]:
#Inspecting first few rows of dataframe
train_vec.head()

,query,domain,0,1,2,3,4,5,6,7,...,5136,5137,5138,5139,5140,5141,5142,5143,5144,5145
0,what expression would i use to say i love you ...,translate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,can you tell me how to say 'i do not speak muc...,translate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"what is the equivalent of, 'life is good' in f...",translate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"tell me how to say, 'it is a beautiful morning...",translate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"if i were mongolian, how would i say that i am...",translate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#dropping the text queries
train_vec = train_vec.drop('query', axis=1)

In [19]:
#inspecting values of a randomly chosen column to check that not all entries are 0.0
train_vec[5136].value_counts().head(10)

0.000000    15095
0.744097        1
0.731775        1
0.694293        1
0.577886        1
0.698728        1
Name: 5136, dtype: int64

In [20]:
#inspecting unique domains
train_vec['domain'].unique()

array(['translate', 'transfer', 'timer', 'definition', 'meaning_of_life',
       'insurance_change', 'find_phone', 'travel_alert', 'pto_request',
       'improve_credit_score', 'fun_fact', 'change_language', 'payday',
       'replacement_card_duration', 'time', 'application_status',
       'flight_status', 'flip_coin', 'change_user_name',
       'where_are_you_from', 'shopping_list_update', 'what_can_i_ask_you',
       'maybe', 'oil_change_how', 'restaurant_reservation', 'balance',
       'confirm_reservation', 'freeze_account', 'rollover_401k',
       'who_made_you', 'distance', 'user_name', 'timezone', 'next_song',
       'transactions', 'restaurant_suggestion', 'rewards_balance',
       'pay_bill', 'spending_history', 'pto_request_status',
       'credit_score', 'new_card', 'lost_luggage', 'repeat', 'mpg',
       'oil_change_when', 'yes', 'travel_suggestion', 'insurance',
       'todo_list_update', 'reminder', 'change_speed', 'tire_pressure',
       'no', 'apr', 'nutrition_info', 'c

In [21]:
#constructing a mapping dictionary for domains to tranform them into nubmers
y_dic = {}
domain = 0
for item in train_vec['domain'].unique():
    y_dic[item] = domain
    domain += 1

In [22]:
#shuffling training data
train_vec = shuffle(train_vec, random_state=0)
train_vec.head()

,domain,0,1,2,3,4,5,6,7,8,...,5136,5137,5138,5139,5140,5141,5142,5143,5144,5145
8218,do_you_have_pets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8136,routing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,insurance_change,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7446,todo_list,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3978,pto_request_status,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#creating a column of target values
train_vec['y'] = train_vec['domain'].replace(y_dic)
#removing the text domains
train_vec = train_vec.iloc[:, 1:]
#defining x and y data
train_x = train_vec.iloc[:,:-1]
train_y = train_vec.iloc[:,-1]

In [24]:
#Transforming queries from other datasets to vectors
Xv = vectorizer.transform(val['query']).toarray()
Xvo = vectorizer.transform(oos_val['query']).toarray()
Xtest = vectorizer.transform(test['query']).toarray()
Xto = vectorizer.transform(oos_test['query']).toarray()

In [25]:
#turning arrays to dataframes
Xv_df = pd.DataFrame(Xv)
Xvo_df = pd.DataFrame(Xvo)
Xtest_df = pd.DataFrame(Xtest)
Xto_df = pd.DataFrame(Xto)

In [26]:
#creating vector datasets for validation set (excluding out of scope values)
val_vec = pd.concat([val, Xv_df], axis=1)
val_vec = val_vec.drop('query', axis=1)
val_vec = shuffle(val_vec, random_state=0)
val_vec['y'] = val_vec['domain'].replace(y_dic)
val = val_vec.iloc[:, 1:]
val_x = val.iloc[:,:-1]
val_y = val.iloc[:,-1]

In [27]:
#creating vector datasets for out of scope validation set
val_oos_vec = pd.concat([oos_val, Xvo_df], axis=1)
val_oos_vec = val_oos_vec.drop('query', axis=1)
val_oos_vec = shuffle(val_oos_vec, random_state=0)
val_oos_vec['y'] = val_oos_vec['domain'].replace(y_dic)
val_oos = val_oos_vec.iloc[:, 1:]
val_oos_x = val_oos.iloc[:,:-1]
val_oos_y = val_oos.iloc[:,-1]
val_full = val.append(val_oos, ignore_index=True)
val_full_x = val_full.iloc[:,:-1]
val_full_y = val_full.iloc[:,-1]

In [28]:
#creating vector datasets for testing set (excluding out of scope values)
test_vec = pd.concat([test, Xtest_df], axis=1)
test_vec = test_vec.drop('query', axis=1)
test_vec = shuffle(test_vec, random_state=0)
test_vec['y']= test_vec['domain'].replace(y_dic)
test = test_vec.iloc[:, 1:]
test_x = test.iloc[:,:-1]
test_y = test.iloc[:,-1]

In [29]:
#creating vector datasets for out of scope testing set
test_oos_vec = pd.concat([oos_test, Xto_df], axis=1)
test_oos_vec = test_oos_vec.drop('query', axis=1)
test_oos_vec = shuffle(test_oos_vec, random_state=0)
test_oos_vec['y'] = test_oos_vec['domain'].replace(y_dic)
test_oos = test_oos_vec.iloc[:, 1:]
test_oos_x = test.iloc[:,:-1]
test_oos_y = test.iloc[:,-1]

In [30]:
#importing torch packages
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils


Initial MLP using parameters from reference paper.

In [32]:
#defining perceptron dimensions
vocab_dim = 5146 #number of words in the vocabulary
hidden_dim = 400 #number of neurons in hidden layer
output_dim = 151 #number of classes (150 in scope plus Out of Scope)

In [33]:
#assembling training arrays.
train_y = np.array(train_y)
train_x = np.array(train_x)

In [34]:
#defining multilayer perceptron
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim, #setting input dimensions
            hidden_dim=hidden_dim, #setting hidden layer dimensions
            output_dim=output_dim, #setting output dimensions
            dropout=0 #setting a dropout rate
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout) #defining the dropout function

        self.hidden = nn.Linear(input_dim, hidden_dim) #defining the hidden layer function
        self.output = nn.Linear(hidden_dim, output_dim) #defining the output layer function

    def forward(self, X, **kwargs):
        X = torch.tanh(self.hidden(X)) #applying a tanh activation function to outputs from the hidden layer
        X = self.dropout(X) #applying the dropout function
        X = F.softmax(self.output(X), dim=-1) #applying the softmax function to the outputs
        return X

In [35]:
#importing skorch modules
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from skorch.dataset import Dataset
from skorch.helper import predefined_split

In [36]:
#transforming data to tensors
train_x = torch.tensor(train_x).float()
val_full_x = torch.tensor(np.array(val_full_x)).float()
val_x = torch.tensor(np.array(val_x)).float()
val_oos_x = torch.tensor(np.array(val_oos_x)).float()
test_x = torch.tensor(np.array(test_x)).float()
test_oos_x = torch.tensor(np.array(test_oos_x)).float()

In [37]:
#transforming targets to arrays
val_full_y = np.array(val_full_y)
val_y = np.array(val_y)
val_oos_y = np.array(val_oos_y)
test_y = np.array(test_y)
test_oos_y = np.array(test_oos_y)

In [38]:
#initialising the classifier
net = NeuralNetClassifier( #a neural network classifier
    module=CLINCModule, #use the MLP defined previously
    criterion=torch.nn.CrossEntropyLoss, #using Cross Entropy Loss as the loss function, due to being a multiclass classification problem
    max_epochs=1000, #defining the maximum number of epochs
    optimizer=torch.optim.SGD, #using stochastic gradient descent for optimization
    callbacks=[EarlyStopping(patience=5)], #implementing early stopping with a patience of 5 epochs
)

In [39]:
#fitting the classifier to the training data
net.fit(train_x, train_y)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0046        5.0173  2.8404
      2        5.0173       0.0046        5.0173  2.5806
      3        5.0173       0.0046        5.0173  2.5644
      4        5.0173       0.0046        5.0173  2.5759
      5        5.0173       0.0046        5.0173  2.6285
Stopping since valid_loss has not improved in the last 5 epochs.


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=CLINCModule(
    (dropout): Dropout(p=0, inplace=False)
    (hidden): Linear(in_features=5146, out_features=400, bias=True)
    (output): Linear(in_features=400, out_features=151, bias=True)
  ),
)

In [40]:
tlabels = net.predict(train_x) #using the trained model to predict from training data
tacc = accuracy_score(tlabels, train_y) #comparing predicted domains to actual classes using classification accuracy
print('training accuracy')
print(tacc) #printing the training accuracy
time0 = time.time() # timer
labels = net.predict(val_x) #predicting validation set
acc = accuracy_score(labels, val_y) #calculating accuracy score for validation predictions
time1 = time.time() # timer
print('Val accuracy')
print(acc) #printing validation accuracy
print('pred time')
print(time1-time0) #printing validation prediction time
time2 = time.time() # timer
olabels = net.predict(val_oos_x) # predicting for Out of Scope Validation set
oos_acc = accuracy_score(olabels, val_oos_y) #calculating accuracy score for OOS validation predictions
time3 = time.time() # timer
print('OOS Val Accuracy')
print(oos_acc) #printing OOS validation accuracy
print('OOS pred time')
print(time3-time2) #printing OOS validation prediction time

training accuracy
0.005298013245033113
Val accuracy
0.003
pred time
0.3200225830078125
OOS Val Accuracy
0.23
OOS pred time
0.01287078857421875


Converged too quickly, likely not in global minimum. Will reattempt using different dropouts, learning rates, momentum.

In [41]:
dropouts = [1, 0.5, 0.1] #defining potential dropout values
tacc={} #initialising training accuracy results dictionary
vacc = {} #initialising validation accuracy results dictionary
vtime = {} #initialising validation time dictionary
oacc = {} #initialising OOS Validation accuracy results dictionary
otime = {} #initialising OOS validation time dictionary
for d in dropouts:    #looping over dropout values
    print(d) #print the dropout rate
    class CLINCModule(nn.Module): #define the module, as per above
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=d #looping through this value
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   #initialising the model as above
    net = NeuralNetClassifier(
    module=CLINCModule,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.SGD,
    callbacks=[EarlyStopping(patience=10)], #Bigger early stopping patience of 10 epochs
    )
    
    #fitting model and calculating results
    net.fit(train_x, train_y) #fitting to training data
    tlabels = net.predict(train_x) #predicting from training data
    tacc[d] = accuracy_score(tlabels, train_y) #calculating training accuracy and storing it in dict
    print('training accuracy')
    print(tacc) #printing full training accuracy dict
    time0 = time.time() #timer
    labels = net.predict(val_x) #predicting validation data
    vacc[d] = accuracy_score(labels, val_y) #calculating validation accuracy and storing it in dict
    time1 = time.time() # timer
    vtime[d] = time1-time0 #calculating validation prediction time & storing in dict 
    print('Val accuracy')
    print(vacc) #printing full validation accuracy dict
    print('pred time')
    print(vtime) #printing full validation prediction time dict
    time2 = time.time() #timer
    olabels = net.predict(val_oos_x) #predicting OOS validation data
    oacc[d] = accuracy_score(olabels, val_oos_y) #calculating OOS validation accuracy and storing in dict
    time3 = time.time() #timer
    otime[d]=time3-time2 #calculating OOS validation prediction time & storing in dict
    print('OOS Val Accuracy')
    print(oacc)# printing full OOS validation accuracy dict
    print('OOS pred time')
    print(otime) #printing full OOS validation prediction time dict

1
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0033        5.0173  2.7095
      2        5.0173       0.0033        5.0173  2.5825
      3        5.0173       0.0033        5.0173  2.5890
      4        5.0173       0.0033        5.0173  2.5863
      5        5.0173       0.0033        5.0173  2.5609
      6        5.0173       0.0033        5.0173  2.5750
      7        5.0173       0.0033        5.0173  2.5444
      8        5.0173       0.0033        5.0173  2.5716
      9        5.0173       0.0033        5.0173  2.5837
     10        5.0173       0.0033        5.0173  2.6356
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{1: 0.0035099337748344373}
Val accuracy
{1: 0.006333333333333333}
pred time
{1: 0.2524840831756592}
OOS Val Accuracy
{1: 0.0}
OOS pred time
{1: 0.008231878280639648}
0.5
  epoch    train_loss    valid_acc    valid_loss     dur
---

In [42]:
learning_rates = [10, 5, 1, 0.5, 0.1, 0.01, 0.001, 0.0001] #looking at different learning rates
#emptying results dicts
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
for lr in learning_rates: #looping through different learning rates   
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=0
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr, #looping through different learning rates
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.SGD,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[lr] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[lr] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[d] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[lr] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[lr]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)

10
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0169        5.0173  2.7157
      2        5.0173       0.0348        5.0172  2.6195
      3        5.0172       0.0540        5.0172  2.6411
      4        5.0172       0.0728        5.0172  2.6060
      5        5.0172       0.0954        5.0172  2.5968
      6        5.0172       0.1185        5.0172  2.5800
      7        5.0171       0.1348        5.0171  2.5846
      8        5.0171       0.1513        5.0171  2.6166
      9        5.0171       0.1705        5.0171  2.6094
     10        5.0171       0.1911        5.0171  2.6303
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.21410596026490067}
Val accuracy
{10: 0.19633333333333333}
pred time
{0.1: 0.24280714988708496}
OOS Val Accuracy
{10: 0.0}
OOS pred time
{10: 0.008342981338500977}
5
  epoch    train_loss    valid_acc    valid_loss     dur


      2        5.0173       0.0073        5.0173  2.6848
      3        5.0173       0.0073        5.0173  2.6724
      4        5.0173       0.0073        5.0173  2.6982
      5        5.0173       0.0073        5.0173  2.7871
      6        5.0173       0.0073        5.0173  2.7626
      7        5.0173       0.0073        5.0173  2.7045
      8        5.0173       0.0073        5.0173  2.6944
      9        5.0173       0.0073        5.0173  2.7003
     10        5.0173       0.0073        5.0173  2.6909
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.21410596026490067, 5: 0.08827814569536424, 1: 0.01695364238410596, 0.5: 0.00814569536423841, 0.1: 0.004172185430463576, 0.01: 0.004503311258278146, 0.001: 0.008013245033112583}
Val accuracy
{10: 0.19633333333333333, 5: 0.084, 1: 0.018, 0.5: 0.009666666666666667, 0.1: 0.0033333333333333335, 0.01: 0.006666666666666667, 0.001: 0.007}
pred time
{0.1: 0.2646620273590088}
OOS Val Accuracy
{10: 0.0, 

In [43]:
momentum = [10, 5, 1, 0.5, 0.1] #changing momentum applied 
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
for p in momentum:    #looping through momentum
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=0
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    optimizer__momentum=p, #looping through momentum values
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.SGD,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[p] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[p] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[d] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[p] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[p]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.0066           nan  2.9037
      2           nan       0.0066           nan  2.8744
      3           nan       0.0066           nan  2.8993
      4           nan       0.0066           nan  2.8462
      5           nan       0.0066           nan  2.8437
      6           nan       0.0066           nan  2.8491
      7           nan       0.0066           nan  2.8397
      8           nan       0.0066           nan  2.8402
      9           nan       0.0066           nan  2.8399
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.006622516556291391}
Val accuracy
{10: 0.006666666666666667}
pred time
{0.1: 0.24516987800598145}
OOS Val Accuracy
{10: 0.0}
OOS pred time
{10: 0.008492708206176758}
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
   

Not converging satisfactorily with SGD. Attempt using Adam optimizer, varying dropout rate.

In [44]:
dropouts = [1, 0.5, 0.1, 0.01, 0.001, 0] #dropouts to try
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
for d in dropouts:    #looping through dropouts
    print(d) #print dropout rate
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=d #dropout rate set to value in loop
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[d] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[d] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[d] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[d] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[d]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)

1
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0053        5.0173  3.7077
      2        5.0173       0.0066        5.0173  3.6635
      3        5.0173       0.0066        5.0173  3.5737
      4        5.0173       0.0066        5.0173  3.7169
      5        5.0173       0.0066        5.0173  3.8467
      6        5.0173       0.0066        5.0173  3.9625
      7        5.0173       0.0066        5.0173  3.8979
      8        5.0173       0.0066        5.0173  3.6462
      9        5.0173       0.0066        5.0173  3.6076
     10        5.0173       0.0066        5.0173  3.6912
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{1: 0.006622516556291391}
Val accuracy
{1: 0.006666666666666667}
pred time
{1: 0.24365901947021484}
OOS Val Accuracy
{1: 0.0}
OOS pred time
{1: 0.00878596305847168}
0.5
  epoch    train_loss    valid_acc    valid_loss     dur
----

     12        4.0427       0.9225        4.1119  3.6072
     13        4.0422       0.9175        4.1157  3.5930
     14        4.0424       0.9185        4.1142  3.6085
     15        4.0398       0.9248        4.1110  3.6459
     16        4.0403       0.9099        4.1234  3.6318
     17        4.0435       0.9083        4.1214  3.5218
     18        4.0419       0.9083        4.1232  3.7210
     19        4.0425       0.9046        4.1272  3.5768
     20        4.0418       0.9083        4.1231  3.6162
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{1: 0.006622516556291391, 0.5: 0.9793377483443708, 0.1: 0.9733112582781457, 0.01: 0.9740397350993377, 0.001: 0.9733774834437086}
Val accuracy
{1: 0.006666666666666667, 0.5: 0.8776666666666667, 0.1: 0.8683333333333333, 0.01: 0.8613333333333333, 0.001: 0.874}
pred time
{1: 0.24365901947021484, 0.5: 0.27951478958129883, 0.1: 0.26523923873901367, 0.01: 0.2642199993133545, 0.001: 0.25611376762390137}
OOS 

This has improved, will retain Adam & dropout rate of 0.5. Now investigating Learning Rates.

In [50]:
learning_rates = [10, 5, 1, 0.5, 0.1, 0.01, 0.001, 1e-4, 1e-5 ,1e-6]
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
for lr in learning_rates:    
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=0.5
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[lr] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[lr] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[d] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[lr] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[lr]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)

10
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0221       0.0066        5.0220  4.3490
      2        5.0220       0.0066        5.0220  4.3117
      3        5.0220       0.0066        5.0220  4.8271
      4        5.0220       0.0066        5.0220  3.9213
      5        5.0220       0.0066        5.0220  4.2449
      6        5.0220       0.0066        5.0220  4.0830
      7        5.0220       0.0066        5.0220  4.0942
      8        5.0220       0.0066        5.0220  4.3219
      9        5.0220       0.0066        5.0220  4.3452
     10        5.0220       0.0066        5.0220  4.3482
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.006622516556291391}
Val accuracy
{10: 0.006666666666666667}
pred time
{0.001: 0.3723461627960205}
OOS Val Accuracy
{10: 0.0}
OOS pred time
{10: 0.01131582260131836}
5
  epoch    train_loss    valid_acc    valid_loss     du

      6        4.0473       0.9219        4.1131  3.9236
      7        4.0455       0.9262        4.1096  3.8869
      8        4.0406       0.9301        4.1069  3.9216
      9        4.0389       0.9291        4.1062  3.9415
     10        4.0375       0.9275        4.1071  4.1294
     11        4.0379       0.9268        4.1085  4.0735
     12        4.0379       0.9285        4.1062  4.0020
     13        4.0370       0.9195        4.1130  3.8344
     14        4.0378       0.9252        4.1087  3.6418
     15        4.0371       0.9248        4.1071  3.7036
     16        4.0368       0.9245        4.1077  3.7419
     17        4.0368       0.9215        4.1112  3.9115
     18        4.0370       0.9255        4.1061  3.9666
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.006622516556291391, 5: 0.006622516556291391, 1: 0.006622516556291391, 0.5: 0.012781456953642384, 0.1: 0.47258278145695365, 0.01: 0.9788741721854305}
Val accuracy
{10: 0

     92        4.0309       0.9377        4.0965  4.4242
     93        4.0309       0.9377        4.0966  4.4377
     94        4.0309       0.9384        4.0962  4.3801
     95        4.0308       0.9371        4.0965  4.4224
     96        4.0308       0.9384        4.0966  4.4281
     97        4.0308       0.9381        4.0966  4.3827
     98        4.0308       0.9391        4.0964  4.4118
     99        4.0308       0.9387        4.0965  4.4504
    100        4.0308       0.9394        4.0964  4.3556
    101        4.0308       0.9374        4.0966  4.3739
    102        4.0308       0.9387        4.0967  4.5497
    103        4.0308       0.9387        4.0966  4.3953
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.006622516556291391, 5: 0.006622516556291391, 1: 0.006622516556291391, 0.5: 0.012781456953642384, 0.1: 0.47258278145695365, 0.01: 0.9788741721854305, 0.001: 0.9859602649006622}
Val accuracy
{10: 0.006666666666666667, 5: 0.0066

     82        4.0771       0.9328        4.1486  3.6110
     83        4.0759       0.9325        4.1477  3.6947
     84        4.0747       0.9328        4.1468  3.6522
     85        4.0740       0.9325        4.1458  3.6171
     86        4.0729       0.9331        4.1449  3.6124
     87        4.0721       0.9328        4.1440  3.5807
     88        4.0705       0.9328        4.1431  3.5693
     89        4.0693       0.9331        4.1422  3.5716
     90        4.0690       0.9341        4.1413  3.5493
     91        4.0675       0.9341        4.1404  3.5730
     92        4.0665       0.9344        4.1396  3.5952
     93        4.0659       0.9344        4.1387  3.6197
     94        4.0647       0.9348        4.1379  3.6179
     95        4.0636       0.9344        4.1371  4.1802
     96        4.0626       0.9351        4.1363  4.6260
     97        4.0622       0.9351        4.1356  4.4660
     98        4.0610       0.9354        4.1348  4.3168
     99        4.0599       0.9

    191        4.0349       0.9424        4.1061  3.9134
    192        4.0347       0.9424        4.1060  4.0978
    193        4.0346       0.9421        4.1059  3.8796
    194        4.0347       0.9421        4.1059  3.5725
    195        4.0345       0.9421        4.1058  3.7760
    196        4.0347       0.9427        4.1057  4.0846
    197        4.0345       0.9424        4.1056  3.9865
    198        4.0345       0.9414        4.1055  3.7321
    199        4.0343       0.9414        4.1055  3.9641
    200        4.0343       0.9417        4.1054  3.7564
    201        4.0342       0.9417        4.1054  3.8173
    202        4.0342       0.9421        4.1052  3.8850
    203        4.0342       0.9414        4.1051  3.6633
    204        4.0341       0.9414        4.1049  3.7514
    205        4.0340       0.9417        4.1048  3.9749
    206        4.0339       0.9424        4.1047  4.0767
    207        4.0339       0.9417        4.1047  4.1772
    208        4.0339       0.9

Val accuracy
{10: 0.006666666666666667, 5: 0.006666666666666667, 1: 0.006666666666666667, 0.5: 0.013333333333333334, 0.1: 0.44733333333333336, 0.01: 0.874, 0.001: 0.8963333333333333, 0.0001: 0.8983333333333333, 1e-05: 0.211, 1e-06: 0.016}
pred time
{0.001: 0.2431046962738037}
OOS Val Accuracy
{10: 0.0, 5: 0.0, 1: 0.0, 0.5: 0.0, 0.1: 0.0, 0.01: 0.07, 0.001: 0.11, 0.0001: 0.14, 1e-05: 0.0, 1e-06: 0.0}
OOS pred time
{10: 0.01131582260131836, 5: 0.008625030517578125, 1: 0.01005697250366211, 0.5: 0.007462978363037109, 0.1: 0.010442256927490234, 0.01: 0.009956121444702148, 0.001: 0.012646913528442383, 0.0001: 0.18695378303527832, 1e-05: 0.008339881896972656, 1e-06: 0.008316993713378906}


Validation accuracy very similar for learning rates of 0.001 and 0.0001. Former chosen due to faster training time. Investigating effect of number of neurons in hidden layer.

In [45]:
lr = 0.001 #using best learning rate from earlier
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_list = [100, 200, 400, 800, 1600] #hidden layer sizes
for hidden_dim in hidden_list:    #looping through hidden layer sizes
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim, #setting hidden layer size
                output_dim=output_dim,
                dropout=0.5
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0166       0.6139        5.0153  1.4640
      2        5.0072       0.5146        4.9842  1.4421
      3        4.8927       0.5685        4.7778  1.5117
      4        4.6625       0.7056        4.5625  1.4323
      5        4.4843       0.7795        4.4313  1.6715
      6        4.3751       0.8275        4.3504  1.6004
      7        4.3020       0.8589        4.2952  1.5017
      8        4.2513       0.8818        4.2584  1.4168
      9        4.2166       0.8964        4.2296  1.4802
     10        4.1852       0.9046        4.2073  1.4061
     11        4.1655       0.9103        4.1918  1.4697
     12        4.1482       0.9129        4.1804  1.4371
     13        4.1346       0.9179        4.1715  1.4536
     14        4.1245       0.9195        4.1644  1.3987
     15        4.1164       0.9212        4.1583  1.4139
     16        4.1083    

      5        4.2802       0.8811        4.2753  2.1377
      6        4.2122       0.8967        4.2296  2.2624
      7        4.1692       0.9119        4.1997  2.1465
      8        4.1415       0.9205        4.1801  2.1145
      9        4.1214       0.9232        4.1670  2.0629
     10        4.1077       0.9235        4.1579  2.1666
     11        4.0958       0.9285        4.1506  2.2052
     12        4.0858       0.9305        4.1451  2.0893
     13        4.0781       0.9331        4.1394  2.0739
     14        4.0700       0.9354        4.1346  2.0583
     15        4.0650       0.9358        4.1309  2.1139
     16        4.0608       0.9374        4.1279  2.0858
     17        4.0576       0.9391        4.1254  2.0596
     18        4.0545       0.9394        4.1234  2.1064
     19        4.0521       0.9404        4.1216  2.1020
     20        4.0503       0.9401        4.1201  2.2215
     21        4.0485       0.9407        4.1189  2.0695
     22        4.0470       0.9

     31        4.0340       0.9417        4.1044  3.4775
     32        4.0338       0.9421        4.1041  3.4732
     33        4.0335       0.9411        4.1035  3.4961
     34        4.0332       0.9411        4.1032  3.4818
     35        4.0331       0.9424        4.1026  3.4845
     36        4.0329       0.9427        4.1022  3.4852
     37        4.0329       0.9427        4.1022  3.4870
     38        4.0329       0.9417        4.1015  3.4766
     39        4.0328       0.9414        4.1014  3.4807
     40        4.0326       0.9414        4.1011  3.4810
     41        4.0326       0.9414        4.1009  3.5063
     42        4.0325       0.9421        4.1010  3.4088
     43        4.0324       0.9417        4.1007  3.4946
     44        4.0323       0.9414        4.1003  3.4977
     45        4.0322       0.9411        4.1002  3.4775
     46        4.0322       0.9407        4.0999  3.5845
     47        4.0322       0.9421        4.0992  3.5168
     48        4.0321       0.9

     39        4.0317       0.9414        4.0995  6.2374
     40        4.0316       0.9401        4.0993  6.2264
     41        4.0316       0.9411        4.0989  6.2515
     42        4.0316       0.9401        4.0989  6.2312
     43        4.0315       0.9407        4.0988  6.2284
     44        4.0315       0.9407        4.0988  6.2701
     45        4.0315       0.9411        4.0983  6.2650
     46        4.0315       0.9421        4.0983  6.2162
     47        4.0314       0.9421        4.0982  6.1493
     48        4.0314       0.9417        4.0983  6.1465
     49        4.0314       0.9411        4.0980  6.2331
     50        4.0312       0.9417        4.0978  6.2109
     51        4.0312       0.9424        4.0976  6.2606
     52        4.0313       0.9424        4.0974  6.2269
     53        4.0312       0.9427        4.0973  6.2587
     54        4.0312       0.9417        4.0970  6.2244
     55        4.0312       0.9417        4.0968  6.3281
     56        4.0312       0.9

     76        4.0305       0.9404        4.0957  11.0205
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{100: 0.9843708609271523, 200: 0.9852980132450331, 400: 0.9864238410596027, 800: 0.9868211920529801, 1600: 0.9867549668874173}
Val accuracy
{100: 0.896, 200: 0.899, 400: 0.9, 800: 0.901, 1600: 0.9003333333333333}
pred time
{100: 0.21036410331726074, 200: 0.1925978660583496, 400: 0.31163692474365234, 800: 0.362407922744751, 1600: 0.5692059993743896}
OOS Val Accuracy
{100: 0.09, 200: 0.12, 400: 0.12, 800: 0.12, 1600: 0.12}
OOS pred time
{100: 0.005872011184692383, 200: 0.0051860809326171875, 400: 0.01047205924987793, 800: 0.011768817901611328, 1600: 0.018018007278442383}


Very small increase in validation accuracy with increasing hidden layer size up to 800 (around 0.5% from 89.6% for 100 neurons), but significantly increasing time (near double). Will investigate effect of additional hidden layer, using same layer size for both layers to limit computation.

In [46]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_list = [100, 200, 400, 800] #hidden layer sizes
for hidden_dim in hidden_list:    #looping through hidden layer size
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=0.5
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.hidden2 = nn.Linear(hidden_dim, hidden_dim) #additional hiden layer
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = torch.tanh(self.hidden2(X)) #additional layer
            X = self.dropout(X) #additional dropout layer
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0170       0.2563        5.0161  1.5456
      2        4.9989       0.2695        4.9425  1.4847
      3        4.8555       0.4331        4.7242  1.4527
      4        4.6858       0.5921        4.5562  1.5185
      5        4.5525       0.6838        4.4358  1.4512
      6        4.4529       0.7308        4.3619  1.4932
      7        4.3932       0.7579        4.3205  1.4904
      8        4.3447       0.7801        4.2895  1.4620
      9        4.3109       0.7917        4.2715  1.4609
     10        4.2875       0.7967        4.2599  1.4564
     11        4.2694       0.8063        4.2488  1.4581
     12        4.2509       0.8182        4.2360  1.4457
     13        4.2346       0.8308        4.2220  1.4493
     14        4.2185       0.8401        4.2119  1.4799
     15        4.2047       0.8500        4.2030  1.4940
     16        4.1852    

     37        4.0410       0.9179        4.1172  2.2781
     38        4.0411       0.9175        4.1171  2.2547
     39        4.0410       0.9169        4.1169  2.2403
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{100: 0.9678145695364239, 200: 0.9784768211920529}
Val accuracy
{100: 0.873, 200: 0.885}
pred time
{100: 0.16866612434387207, 200: 0.21779298782348633}
OOS Val Accuracy
{100: 0.07, 200: 0.08}
OOS pred time
{100: 0.004477977752685547, 200: 0.006827116012573242}
0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0109       0.1705        4.9560  4.1834
      2        4.6944       0.7275        4.4184  4.0486
      3        4.3032       0.8589        4.2301  4.2061
      4        4.1745       0.9017        4.1690  4.0524
      5        4.1204       0.9156        4.1438  4.1345
      6        4.0896       0.9219        4.1345  4.1656
      7        4.0745 

Worse accuracy than for one layer for all layer sizes, so will use a single layers. As 100 neurons in a single layer performed quite similarly to larger layers, will investigate effect of smaller hidden layer to increase speed.

In [47]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_list = [25, 50, 75] #hidden layer size
for hidden_dim in hidden_list:    #looping for hidden layer size
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,#setting hidden layer size
                output_dim=output_dim,
                dropout=0.5
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0170       0.1593        5.0167  1.2534
      2        5.0157       0.3493        5.0145  1.0968
      3        5.0111       0.3834        5.0061  1.1054
      4        4.9911       0.3073        4.9709  1.0619
      5        4.9445       0.3970        4.9091  1.0228
      6        4.8859       0.4404        4.8360  1.0695
      7        4.8279       0.4745        4.7697  1.0470
      8        4.7768       0.5109        4.7119  1.0754
      9        4.7325       0.5315        4.6623  1.1561
     10        4.6959       0.5487        4.6220  1.0389
     11        4.6648       0.5649        4.5898  1.0400
     12        4.6389       0.5791        4.5638  1.0533
     13        4.6186       0.5911        4.5418  1.1238
     14        4.5977       0.5993        4.5221  1.0451
     15        4.5792       0.6123        4.5039  1.1313
     16        4.5650    

    102        4.1657       0.8997        4.1473  1.0417
    103        4.1646       0.8997        4.1466  1.0497
    104        4.1627       0.9007        4.1463  1.0408
    105        4.1615       0.8993        4.1460  1.0436
    106        4.1628       0.8990        4.1457  1.0390
    107        4.1581       0.9003        4.1453  1.0346
    108        4.1578       0.8987        4.1450  1.0380
    109        4.1574       0.9003        4.1445  1.0430
    110        4.1603       0.9000        4.1443  1.0448
    111        4.1567       0.8997        4.1441  1.0347
    112        4.1556       0.9007        4.1436  1.2328
    113        4.1544       0.9007        4.1433  1.0396
    114        4.1554       0.9010        4.1430  1.0346
    115        4.1554       0.8997        4.1428  1.0308
    116        4.1521       0.9010        4.1426  1.0315
    117        4.1521       0.9007        4.1423  1.0341
    118        4.1519       0.8990        4.1423  1.0279
    119        4.1486       0.8

     19        4.1946       0.8811        4.2033  1.1530
     20        4.1885       0.8834        4.1971  1.1529
     21        4.1798       0.8848        4.1925  1.1491
     22        4.1739       0.8854        4.1883  1.1520
     23        4.1681       0.8861        4.1843  1.1820
     24        4.1629       0.8848        4.1808  1.1494
     25        4.1569       0.8877        4.1769  1.1473
     26        4.1522       0.8930        4.1724  1.1567
     27        4.1456       0.8967        4.1675  1.1580
     28        4.1410       0.8974        4.1642  1.1575
     29        4.1365       0.8997        4.1614  1.1503
     30        4.1325       0.9023        4.1588  1.1540
     31        4.1274       0.9033        4.1566  1.1560
     32        4.1239       0.9030        4.1550  1.1564
     33        4.1205       0.9036        4.1535  1.2002
     34        4.1185       0.9043        4.1520  1.1707
     35        4.1157       0.9046        4.1509  1.2413
     36        4.1126       0.9

      9        4.2895       0.8483        4.2847  1.3191
     10        4.2563       0.8639        4.2581  1.3010
     11        4.2287       0.8715        4.2385  1.3066
     12        4.2075       0.8791        4.2229  1.2926
     13        4.1916       0.8897        4.2095  1.3463
     14        4.1756       0.8944        4.1974  1.3234
     15        4.1623       0.9013        4.1871  1.3014
     16        4.1498       0.9060        4.1779  1.3060
     17        4.1394       0.9116        4.1710  1.3210
     18        4.1311       0.9136        4.1646  1.3380
     19        4.1220       0.9185        4.1586  1.3046
     20        4.1159       0.9199        4.1537  1.3277
     21        4.1089       0.9222        4.1501  1.3110
     22        4.1021       0.9242        4.1472  1.2969
     23        4.0968       0.9255        4.1443  1.3022
     24        4.0930       0.9255        4.1419  1.3230
     25        4.0891       0.9252        4.1398  1.2921
     26        4.0848       0.9

Definitely worse than 100 neurons. Would 151 hidden neurons be beneficial, given output size?

In [48]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_list = [151]
for hidden_dim in hidden_list:    
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=0.5
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = torch.tanh(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0164       0.6871        5.0142  1.8697
      2        4.9909       0.4632        4.9248  1.8631
      3        4.7729       0.6576        4.6273  1.8291
      4        4.5074       0.7858        4.4366  1.8232
      5        4.3543       0.8497        4.3283  1.8285
      6        4.2655       0.8811        4.2671  1.8184
      7        4.2122       0.8934        4.2314  1.8413
      8        4.1786       0.9030        4.2078  1.8570
      9        4.1546       0.9139        4.1886  1.8207
     10        4.1356       0.9189        4.1739  1.8298
     11        4.1188       0.9225        4.1636  1.8285
     12        4.1071       0.9248        4.1558  1.8272
     13        4.0970       0.9252        4.1501  1.8286
     14        4.0882       0.9295        4.1453  1.8223
     15        4.0819       0.9308        4.1413  1.8230
     16        4.0773    

No. 800 neurons has given best accuracy accuracy at the moment. Can this be improved with different activation functions for the hidden layer?

In [49]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X)) #ReLU activation function for hidden layer
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam,
callbacks=[EarlyStopping(patience=10)],
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[hidden_dim] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[hidden_dim] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[hidden_dim] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[hidden_dim]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)
print('ReLU')

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0130       0.2997        4.9847  6.2996
      2        4.7681       0.6013        4.5543  6.2730
      3        4.4382       0.7411        4.3775  6.2243
      4        4.3059       0.8113        4.2919  6.2451
      5        4.2359       0.8305        4.2508  6.2227
      6        4.1975       0.8447        4.2265  6.2319
      7        4.1729       0.8632        4.2091  6.2383
      8        4.1431       0.8987        4.1805  6.1958
      9        4.1115       0.9116        4.1579  6.2770
     10        4.0936       0.9156        4.1471  6.1687
     11        4.0848       0.9166        4.1420  6.2382
     12        4.0799       0.9162        4.1392  6.2232
     13        4.0758       0.9192        4.1377  6.2241
     14        4.0685       0.9242        4.1334  6.1465
     15        4.0594       0.9308        4.1262  6.1324
     16        4.0530       0.9

Slight improvement with ReLU (0.3%). Will attempt other functions to see whether they outperform ReLU, otherwise keep. Sigmoid:

In [50]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.sigmoid(self.hidden(X)) #sigmoid activation function
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam,
callbacks=[EarlyStopping(patience=10)],
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[hidden_dim] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[hidden_dim] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[hidden_dim] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[hidden_dim]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)
print('sigmoid')

/Users/ollie/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0180       0.0066        5.0166  6.3491
      2        5.0165       0.0232        5.0146  6.3888
      3        5.0135       0.0344        5.0097  6.4449
      4        5.0064       0.0576        4.9979  6.3237
      5        4.9930       0.0679        4.9813  6.2590
      6        4.9763       0.0772        4.9647  6.3531
      7        4.9621       0.0768        4.9568  6.2890
      8        4.9560       0.0778        4.9533  6.2982
      9        4.9528       0.0808        4.9503  6.2021
     10        4.9486       0.0874        4.9457  6.3159
     11        4.9428       0.0954        4.9393  6.2446
     12        4.9365       0.1036        4.9321  6.2184
     13        4.9292       0.1175        4.9257  6.2237
     14        4.9233       0.1252        4.9166  6.2313
     15        4.9119       0.1358        4.9038  6.2675
     16        4.9017       0.1

     99        4.4890       0.5397        4.5048  6.1875
    100        4.4811       0.5430        4.4979  6.1579
    101        4.4782       0.5440        4.4963  6.2313
    102        4.4745       0.5450        4.4946  6.2708
    103        4.4709       0.5536        4.4893  6.1081
    104        4.4665       0.5490        4.4894  6.2632
    105        4.4626       0.5570        4.4842  6.3135
    106        4.4574       0.5649        4.4805  6.5914
    107        4.4534       0.5679        4.4777  6.9639
    108        4.4498       0.5712        4.4723  6.9868
    109        4.4454       0.5758        4.4709  6.9394
    110        4.4409       0.5772        4.4665  7.0651
    111        4.4377       0.5785        4.4619  7.1449
    112        4.4352       0.5805        4.4593  7.1906
    113        4.4343       0.5821        4.4581  6.4159
    114        4.4327       0.5841        4.4569  6.2426
    115        4.4297       0.5874        4.4524  6.2408
    116        4.4276       0.5

    203        4.2315       0.7553        4.2829  6.2813
    204        4.2279       0.7646        4.2717  6.3001
    205        4.2254       0.7675        4.2733  6.3484
    206        4.2227       0.7662        4.2731  5.8018
    207        4.2219       0.7709        4.2678  6.4278
    208        4.2206       0.7715        4.2684  6.2424
    209        4.2201       0.7692        4.2688  8.5317
    210        4.2193       0.7705        4.2686  8.0056
    211        4.2175       0.7742        4.2653  6.9680
    212        4.2154       0.7735        4.2653  6.4788
    213        4.2140       0.7755        4.2637  6.5176
    214        4.2142       0.7785        4.2607  6.5550
    215        4.2134       0.7748        4.2623  6.6556
    216        4.2131       0.7728        4.2622  6.6729
    217        4.2128       0.7778        4.2602  6.6288
    218        4.2122       0.7801        4.2586  6.3800
    219        4.2117       0.7785        4.2582  6.4649
    220        4.2116       0.7

    313        4.0988       0.8742        4.1639  6.7759
    314        4.0990       0.8705        4.1657  7.0871
    315        4.0987       0.8735        4.1638  6.6992
    316        4.0989       0.8689        4.1655  6.3662
    317        4.0981       0.8722        4.1631  6.3561
    318        4.0985       0.8712        4.1636  6.3542
    319        4.0986       0.8748        4.1618  6.3413
    320        4.0980       0.8712        4.1617  6.5516
    321        4.0979       0.8719        4.1627  6.5108
    322        4.0981       0.8722        4.1611  6.7265
    323        4.0980       0.8738        4.1612  6.6053
    324        4.0980       0.8705        4.1625  6.6099
    325        4.0981       0.8768        4.1605  6.6186
    326        4.0979       0.8725        4.1624  6.5841
    327        4.0971       0.8752        4.1613  6.7656
    328        4.0935       0.8805        4.1572  6.7401
    329        4.0929       0.8765        4.1590  6.6377
    330        4.0923       0.8

In [51]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = self.hidden(X) #no activation function (linear)
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam,
callbacks=[EarlyStopping(patience=10)],
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[hidden_dim] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[hidden_dim] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[hidden_dim] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[hidden_dim]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0089       0.5252        4.9527  6.3222
      2        4.6487       0.8070        4.3973  6.4461
      3        4.2653       0.8957        4.2314  6.8577
      4        4.1554       0.9199        4.1781  6.8629
      5        4.1109       0.9288        4.1555  6.5649
      6        4.0846       0.9328        4.1418  6.4810
      7        4.0674       0.9394        4.1323  6.7712
      8        4.0572       0.9394        4.1263  6.4031
      9        4.0510       0.9414        4.1221  6.3516
     10        4.0465       0.9411        4.1194  6.3315
     11        4.0436       0.9427        4.1166  6.3509
     12        4.0414       0.9437        4.1146  6.3308
     13        4.0398       0.9424        4.1125  6.3491
     14        4.0385       0.9437        4.1109  6.2971
     15        4.0371       0.9447        4.1091  6.2933
     16        4.0360       0.9

ReLU found to be best. Re-iterating through variables previously optimised for tanh activation. Dropouts:

In [52]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
dropout_list = [0.75, 0.5, 0.25, 0.1, 0.01] #adjusting dropout rate
for dropout in dropout_list:    #looping for dropouts
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=dropout #setting dropout rate
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[dropout] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[dropout] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[dropout] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[dropout] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[dropout]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.4315        5.0101  6.6061
      2        4.9120       0.4679        4.7299  6.5239
      3        4.6053       0.6715        4.4840  6.4711
      4        4.4244       0.7596        4.3682  6.2988
      5        4.3288       0.7964        4.3018  6.3068
      6        4.2741       0.8225        4.2652  6.3612
      7        4.2322       0.8543        4.2316  6.3762
      8        4.1990       0.8689        4.2087  6.3063
      9        4.1730       0.8798        4.1914  6.2668
     10        4.1551       0.8987        4.1729  6.2441
     11        4.1317       0.9073        4.1595  6.2421
     12        4.1193       0.9073        4.1520  6.2562
     13        4.1092       0.9132        4.1462  6.2718
     14        4.0980       0.9189        4.1392  6.2936
     15        4.0893       0.9298        4.1322  6.3823
     16        4.0811    

      6        4.1918       0.8583        4.2205  6.7897
      7        4.1625       0.8778        4.1984  6.3732
      8        4.1357       0.8947        4.1771  6.6081
      9        4.1153       0.9036        4.1650  6.4892
     10        4.0983       0.9096        4.1528  6.6590
     11        4.0899       0.9126        4.1469  6.5947
     12        4.0856       0.9113        4.1456  6.4069
     13        4.0804       0.9156        4.1404  6.2320
     14        4.0745       0.9146        4.1415  6.1545
     15        4.0693       0.9212        4.1359  6.1887
     16        4.0600       0.9262        4.1274  6.1672
     17        4.0554       0.9321        4.1217  6.1663
     18        4.0495       0.9331        4.1175  6.1480
     19        4.0463       0.9348        4.1148  6.2277
     20        4.0449       0.9358        4.1133  6.2520
     21        4.0440       0.9387        4.1114  6.1674
     22        4.0431       0.9371        4.1103  6.1559
     23        4.0424       0.9

     34        4.0317       0.9411        4.1002  6.3346
     35        4.0317       0.9421        4.0996  6.3608
     36        4.0315       0.9407        4.0997  6.2042
     37        4.0315       0.9417        4.0991  6.3066
     38        4.0314       0.9427        4.0987  6.3781
     39        4.0313       0.9417        4.0987  6.3579
     40        4.0313       0.9414        4.0982  6.4128
     41        4.0313       0.9427        4.0983  6.2995
     42        4.0312       0.9434        4.0981  6.4869
     43        4.0311       0.9444        4.0975  6.6551
     44        4.0311       0.9430        4.0972  6.7829
     45        4.0311       0.9417        4.0973  6.8611
     46        4.0310       0.9430        4.0971  6.5274
     47        4.0309       0.9434        4.0973  6.2378
     48        4.0310       0.9447        4.0967  6.1882
     49        4.0309       0.9424        4.0961  6.1774
     50        4.0309       0.9424        4.0965  6.1851
     51        4.0309       0.9

     70        4.0305       0.9407        4.0958  6.2372
     71        4.0306       0.9417        4.0955  6.2266
     72        4.0304       0.9424        4.0956  6.2432
     73        4.0304       0.9411        4.0959  6.2296
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{0.75: 0.9866887417218543, 0.5: 0.9866887417218543, 0.25: 0.9873509933774834, 0.1: 0.9871523178807947}
Val accuracy
{0.75: 0.9066666666666666, 0.5: 0.9053333333333333, 0.25: 0.9036666666666666, 0.1: 0.9056666666666666}
pred time
{0.75: 0.49721789360046387, 0.5: 0.42409491539001465, 0.25: 0.37950801849365234, 0.1: 0.3557312488555908}
OOS Val Accuracy
{0.75: 0.18, 0.5: 0.2, 0.25: 0.27, 0.1: 0.27}
OOS pred time
{0.75: 0.014636039733886719, 0.5: 0.014017105102539062, 0.25: 0.012434005737304688, 0.1: 0.012111186981201172}
0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0026       0.3546        4.8

In [53]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
dropout_list = [1, 0.9, 0.8, 0] #some further values for dropout
for dropout in dropout_list:    
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[dropout] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[dropout] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[dropout] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[dropout] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[dropout]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0070        5.0173  6.2591
      2        5.0173       0.0060        5.0173  6.2635
      3        5.0173       0.0073        5.0173  6.2483
      4        5.0173       0.0063        5.0173  6.2667
      5        5.0173       0.0066        5.0173  6.2426
      6        5.0173       0.0066        5.0173  6.2644
      7        5.0173       0.0066        5.0173  6.2378
      8        5.0173       0.0066        5.0173  6.2465
      9        5.0173       0.0066        5.0173  6.2433
     10        5.0173       0.0066        5.0173  6.2304
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{1: 0.006622516556291391}
Val accuracy
{1: 0.006666666666666667}
pred time
{1: 0.37184715270996094}
OOS Val Accuracy
{1: 0.0}
OOS pred time
{1: 0.01224207878112793}
0.001
  epoch    train_loss    valid_acc    valid_loss     du

     97        4.0419       0.9417        4.0944  6.5697
     98        4.0419       0.9421        4.0941  6.7414
     99        4.0407       0.9437        4.0941  6.6712
    100        4.0404       0.9440        4.0942  6.7380
    101        4.0408       0.9434        4.0941  6.5589
    102        4.0407       0.9444        4.0941  6.6832
    103        4.0402       0.9427        4.0941  6.6383
    104        4.0396       0.9427        4.0938  6.6133
    105        4.0395       0.9404        4.0940  6.6393
    106        4.0399       0.9414        4.0932  6.7592
    107        4.0396       0.9414        4.0935  6.7308
    108        4.0402       0.9421        4.0930  6.6406
    109        4.0396       0.9417        4.0936  6.9146
    110        4.0397       0.9440        4.0936  6.6158
    111        4.0394       0.9414        4.0938  6.5966
    112        4.0392       0.9394        4.0939  6.5071
    113        4.0398       0.9407        4.0938  6.5118
    114        4.0391       0.9

Val accuracy
{1: 0.006666666666666667, 0.9: 0.9053333333333333, 0.8: 0.9046666666666666}
pred time
{1: 0.37184715270996094, 0.9: 0.3441929817199707, 0.8: 0.36145877838134766}
OOS Val Accuracy
{1: 0.0, 0.9: 0.21, 0.8: 0.2}
OOS pred time
{1: 0.01224207878112793, 0.9: 0.01036524772644043, 0.8: 0.01233983039855957}
0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0019       0.3301        4.8913  6.1668
      2        4.6103       0.6781        4.4432  6.1750
      3        4.3458       0.7884        4.3179  6.2022
      4        4.2375       0.8222        4.2582  6.2215
      5        4.1896       0.8384        4.2339  6.2444
      6        4.1660       0.8606        4.2146  6.2118
      7        4.1396       0.8825        4.1958  6.2041
      8        4.1135       0.8911        4.1750  6.2062
      9        4.0982       0.8980        4.1637  6.1601
     10        4.0895       0.9089        4.1540  6.18

    104        4.0302       0.9421        4.0930  6.1516
    105        4.0302       0.9421        4.0930  6.1369
    106        4.0302       0.9421        4.0929  6.2087
    107        4.0302       0.9424        4.0929  6.1217
    108        4.0302       0.9424        4.0928  6.1294
    109        4.0302       0.9427        4.0928  6.2150
    110        4.0302       0.9424        4.0927  6.1270
    111        4.0302       0.9424        4.0927  6.1530
    112        4.0302       0.9417        4.0926  6.1249
    113        4.0302       0.9417        4.0926  6.1259
    114        4.0302       0.9414        4.0925  6.1312
    115        4.0302       0.9417        4.0925  6.1283
    116        4.0302       0.9414        4.0925  6.1311
    117        4.0302       0.9421        4.0924  6.1440
    118        4.0302       0.9417        4.0924  6.3403
    119        4.0302       0.9421        4.0923  6.2250
    120        4.0302       0.9417        4.0923  5.9141
Stopping since valid_loss has n

0.75 found to be most accurate, but not much in it. Will adjust learning rate adjustment by factor of 10 either way:

In [54]:
learning_rates = [0.01, 0.0001]
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
dropout = 0.75
for lr in learning_rates:    
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[lr] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[lr] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[lr] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[lr] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[lr]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.01
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.7507       0.7474        4.3177  6.5751
      2        4.2561       0.8609        4.1816  6.1611
      3        4.1629       0.8831        4.1560  6.1635
      4        4.1261       0.9063        4.1288  6.2263
      5        4.1001       0.9225        4.1133  6.1903
      6        4.0816       0.9242        4.1082  6.2384
      7        4.0693       0.9248        4.1080  6.2755
      8        4.0661       0.9275        4.1060  6.1246
      9        4.0610       0.9281        4.1045  6.2033
     10        4.0602       0.9298        4.1043  6.2742
     11        4.0562       0.9255        4.1049  6.3306
     12        4.0548       0.9258        4.1054  6.3616
     13        4.0536       0.9291        4.1029  6.4214
     14        4.0537       0.9288        4.1029  6.3477
     15        4.0526       0.9308        4.1013  6.3679
     16        4.0495     

     69        4.2469       0.8238        4.2732  6.1097
     70        4.2434       0.8255        4.2695  6.1337
     71        4.2416       0.8265        4.2666  6.1335
     72        4.2362       0.8265        4.2639  6.2552
     73        4.2341       0.8281        4.2616  6.0734
     74        4.2316       0.8281        4.2592  6.1118
     75        4.2277       0.8291        4.2569  6.2110
     76        4.2262       0.8305        4.2544  6.1312
     77        4.2221       0.8354        4.2517  6.1991
     78        4.2196       0.8364        4.2496  6.1359
     79        4.2167       0.8368        4.2477  6.1426
     80        4.2151       0.8394        4.2455  6.1577
     81        4.2129       0.8417        4.2429  6.1455
     82        4.2098       0.8424        4.2407  6.1397
     83        4.2071       0.8430        4.2385  6.1501
     84        4.2049       0.8450        4.2362  6.1419
     85        4.2021       0.8467        4.2341  6.1299
     86        4.1987       0.8

    172        4.0836       0.9185        4.1401  6.2262
    173        4.0832       0.9179        4.1396  6.3218
    174        4.0822       0.9189        4.1393  6.2505
    175        4.0819       0.9185        4.1390  6.2359
    176        4.0811       0.9195        4.1386  6.2313
    177        4.0809       0.9192        4.1382  6.2133
    178        4.0811       0.9199        4.1378  6.2251
    179        4.0793       0.9195        4.1374  6.5190
    180        4.0792       0.9192        4.1371  6.4926
    181        4.0792       0.9205        4.1367  6.3302
    182        4.0784       0.9202        4.1364  6.3319
    183        4.0785       0.9199        4.1362  6.3877
    184        4.0779       0.9205        4.1358  6.3466
    185        4.0776       0.9205        4.1354  6.2544
    186        4.0763       0.9205        4.1351  6.2174
    187        4.0770       0.9202        4.1347  6.2368
    188        4.0759       0.9215        4.1343  6.2280
    189        4.0752       0.9

    284        4.0504       0.9318        4.1130  6.7190
    285        4.0496       0.9315        4.1128  6.5241
    286        4.0500       0.9311        4.1125  6.7529
    287        4.0497       0.9308        4.1124  6.4807
    288        4.0497       0.9308        4.1123  6.5640
    289        4.0497       0.9311        4.1122  6.5109
    290        4.0498       0.9308        4.1122  6.5659
    291        4.0497       0.9305        4.1120  6.4801
    292        4.0497       0.9315        4.1118  6.5687
    293        4.0492       0.9318        4.1117  6.4580
    294        4.0494       0.9325        4.1115  6.5823
    295        4.0492       0.9325        4.1116  6.5719
    296        4.0490       0.9325        4.1114  6.5568
    297        4.0491       0.9328        4.1114  6.5844
    298        4.0488       0.9321        4.1113  6.6224
    299        4.0487       0.9338        4.1107  6.4403
    300        4.0480       0.9354        4.1094  6.4622
    301        4.0464       0.9

learning rate of 0.001 remains most accurate. Attempting SGD as optimizer again to see if that can improve on where we are:

In [56]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800
dropout = 0.75
momentum_list = [10, 1, 0.1, 0.01, 0.001, 0.0001] #assorted momentums for this learning rate
for p in momentum_list:    
    print(lr)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.SGD, #using SGD as optimizer
    optimizer__momentum = p, #looping through momentums
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[p] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[p] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[p] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[p] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[p]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.0066           nan  5.0247
      2           nan       0.0066           nan  5.0387
      3           nan       0.0066           nan  4.9342
      4           nan       0.0066           nan  4.8973
      5           nan       0.0066           nan  4.9366
      6           nan       0.0066           nan  4.9376
      7           nan       0.0066           nan  4.9249
      8           nan       0.0066           nan  4.9357
      9           nan       0.0066           nan  5.0245
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{10: 0.006622516556291391}
Val accuracy
{10: 0.006666666666666667}
pred time
{10: 0.35844969749450684}
OOS Val Accuracy
{10: 0.0}
OOS pred time
{10: 0.012405633926391602}
0.001
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------ 

Not converging. Adam retained. Investigating hidden layer size again to confirm 800 optimal, including much larger hidden layer:

In [57]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim_list = [50, 100, 200, 400, 800, 1600, 3200, 6400, 12800] #hidden layer size
dropout = 0.75
for hidden_dim in hidden_dim_list:    #looping for hidden layer size
    print(hidden_dim)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim, #setting hidden layer size
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam, #Adam again - momentum has been removed accordingly.
    callbacks=[EarlyStopping(patience=10)],
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

50
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0171       0.1003        5.0168  1.2308
      2        5.0161       0.3182        5.0151  1.1453
      3        5.0126       0.2010        5.0086  1.1499
      4        4.9988       0.2308        4.9795  1.1425
      5        4.9725       0.2950        4.9310  1.1498
      6        4.9466       0.3500        4.8811  1.2321
      7        4.9217       0.3828        4.8349  1.1786
      8        4.8983       0.4262        4.7910  1.2794
      9        4.8830       0.4583        4.7538  1.3499
     10        4.8646       0.4917        4.7219  1.1764
     11        4.8473       0.5169        4.6907  1.2365
     12        4.8351       0.5424        4.6624  1.2033
     13        4.8219       0.5540        4.6364  1.2312
     14        4.8108       0.5649        4.6119  1.3854
     15        4.8028       0.5791        4.5886  1.4049
     16        4.7873       

    102        4.5083       0.8411        4.2172  1.2439
    103        4.5121       0.8421        4.2159  1.2251
    104        4.5134       0.8417        4.2147  1.2163
    105        4.5107       0.8424        4.2137  1.2485
    106        4.5149       0.8430        4.2122  1.2390
    107        4.5093       0.8434        4.2115  1.2381
    108        4.5112       0.8437        4.2107  1.2478
    109        4.5036       0.8427        4.2100  1.3385
    110        4.5125       0.8434        4.2099  1.3001
    111        4.5014       0.8440        4.2091  1.2332
    112        4.5058       0.8437        4.2084  1.2357
    113        4.4964       0.8467        4.2074  1.2409
    114        4.5010       0.8457        4.2069  1.2552
    115        4.4974       0.8467        4.2069  1.2314
    116        4.4939       0.8470        4.2060  1.2482
    117        4.4905       0.8487        4.2050  1.2414
    118        4.5001       0.8487        4.2043  1.2749
    119        4.5014       0.8

    220        4.4140       0.8629        4.1789  1.2701
    221        4.4147       0.8632        4.1789  1.2620
    222        4.4166       0.8613        4.1791  1.2601
    223        4.4100       0.8619        4.1793  1.2705
    224        4.4191       0.8609        4.1795  1.2652
    225        4.4098       0.8609        4.1793  1.2639
    226        4.4099       0.8623        4.1793  1.2722
    227        4.4113       0.8619        4.1794  1.2683
    228        4.4084       0.8606        4.1791  1.2691
Stopping since valid_loss has not improved in the last 10 epochs.
training accuracy
{50: 0.9144370860927152}
Val accuracy
{50: 0.8266666666666667}
pred time
{50: 0.1441209316253662}
OOS Val Accuracy
{50: 0.04}
OOS pred time
{50: 0.0038161277770996094}
100
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0170       0.2689        5.0165  1.4337
      2        5.0149       0.4285        5.0122  1.4186
   

     93        4.1908       0.8834        4.1580  1.5022
     94        4.1923       0.8848        4.1573  1.4859
     95        4.1918       0.8848        4.1567  1.4935
     96        4.1887       0.8834        4.1564  1.4932
     97        4.1886       0.8848        4.1557  1.4913
     98        4.1898       0.8838        4.1558  1.4883
     99        4.1853       0.8828        4.1551  1.4912
    100        4.1853       0.8821        4.1550  1.5107
    101        4.1830       0.8828        4.1544  1.4851
    102        4.1819       0.8844        4.1538  1.6649
    103        4.1819       0.8848        4.1537  1.5060
    104        4.1774       0.8844        4.1529  1.5476
    105        4.1771       0.8861        4.1510  1.4912
    106        4.1784       0.8911        4.1467  1.4934
    107        4.1747       0.8930        4.1447  1.4998
    108        4.1721       0.8921        4.1448  1.4959
    109        4.1724       0.8921        4.1448  1.5948
    110        4.1723       0.8

     44        4.1112       0.9113        4.1337  2.2582
     45        4.1104       0.9113        4.1329  2.1436
     46        4.1095       0.9139        4.1322  2.1699
     47        4.1066       0.9139        4.1319  2.2201
     48        4.1041       0.9159        4.1303  2.2653
     49        4.1006       0.9139        4.1296  2.2250
     50        4.0974       0.9129        4.1283  2.1840
     51        4.0977       0.9162        4.1273  2.2352
     52        4.0953       0.9189        4.1247  2.3422
     53        4.0921       0.9209        4.1226  2.5611
     54        4.0895       0.9222        4.1217  2.3258
     55        4.0878       0.9209        4.1214  2.2840
     56        4.0884       0.9205        4.1210  2.4763
     57        4.0846       0.9232        4.1198  2.4433
     58        4.0837       0.9209        4.1197  2.2998
     59        4.0822       0.9215        4.1189  2.3312
     60        4.0820       0.9262        4.1164  2.2978
     61        4.0779       0.9

     18        4.1284       0.9109        4.1496  3.5247
     19        4.1203       0.9129        4.1448  3.5568
     20        4.1127       0.9136        4.1421  3.5397
     21        4.1086       0.9136        4.1397  3.5517
     22        4.1033       0.9199        4.1355  3.5539
     23        4.0992       0.9222        4.1330  3.5776
     24        4.0933       0.9205        4.1318  3.5672
     25        4.0897       0.9212        4.1299  3.5515
     26        4.0876       0.9235        4.1279  3.6154
     27        4.0837       0.9232        4.1272  3.7022
     28        4.0824       0.9212        4.1265  3.8041
     29        4.0798       0.9219        4.1250  3.8737
     30        4.0753       0.9285        4.1230  3.6562
     31        4.0720       0.9305        4.1202  3.5309
     32        4.0689       0.9301        4.1181  3.6022
     33        4.0667       0.9305        4.1177  3.5796
     34        4.0642       0.9311        4.1148  3.5962
     35        4.0616       0.9

     19        4.0673       0.9311        4.1209  6.3732
     20        4.0629       0.9351        4.1187  6.2594
     21        4.0581       0.9348        4.1159  6.2065
     22        4.0552       0.9391        4.1132  6.2833
     23        4.0514       0.9397        4.1109  6.2887
     24        4.0489       0.9427        4.1089  6.4198
     25        4.0472       0.9421        4.1084  6.4481
     26        4.0460       0.9427        4.1067  6.3913
     27        4.0442       0.9421        4.1063  6.2536
     28        4.0434       0.9447        4.1044  6.3409
     29        4.0422       0.9437        4.1039  6.3926
     30        4.0407       0.9417        4.1034  6.3065
     31        4.0407       0.9427        4.1027  6.2962
     32        4.0404       0.9437        4.1024  6.3609
     33        4.0393       0.9450        4.1014  6.3610
     34        4.0386       0.9444        4.0999  6.4921
     35        4.0382       0.9444        4.0998  6.5013
     36        4.0375       0.9

     42        4.0324       0.9470        4.0943  11.1228
     43        4.0320       0.9464        4.0947  11.3458
     44        4.0323       0.9444        4.0948  11.1179
     45        4.0321       0.9450        4.0948  11.1263
     46        4.0320       0.9454        4.0941  11.1258
     47        4.0320       0.9440        4.0942  11.1458
     48        4.0320       0.9467        4.0931  11.2284
     49        4.0318       0.9480        4.0929  11.1924
     50        4.0318       0.9457        4.0934  11.1937
     51        4.0316       0.9460        4.0933  11.0919
     52        4.0317       0.9457        4.0935  11.1520
     53        4.0319       0.9460        4.0937  11.1545
     54        4.0315       0.9454        4.0938  11.1866
     55        4.0317       0.9450        4.0943  11.2692
     56        4.0315       0.9470        4.0938  11.3958
     57        4.0314       0.9460        4.0930  11.2342
Stopping since valid_loss has not improved in the last 10 epochs.
traini

      8        4.0458       0.9381        4.1128  38.6286
      9        4.0406       0.9397        4.1117  36.8542
     10        4.0383       0.9401        4.1078  36.0486
     11        4.0364       0.9421        4.1055  36.7903
     12        4.0356       0.9417        4.1045  39.0225
     13        4.0348       0.9424        4.1027  39.1421
     14        4.0341       0.9437        4.1021  39.2116
     15        4.0338       0.9447        4.1012  40.0323
     16        4.0336       0.9440        4.1015  40.0047
     17        4.0331       0.9414        4.1028  39.6398
     18        4.0330       0.9424        4.1004  39.2430
     19        4.0327       0.9444        4.0996  37.7536
     20        4.0325       0.9421        4.0998  37.4436
     21        4.0322       0.9447        4.0987  37.4940
     22        4.0323       0.9430        4.0995  37.7521
     23        4.0320       0.9434        4.0989  37.5587
     24        4.0321       0.9440        4.0976  37.5614
     25       

Decrease in performance after 800. Experimenting with additional hidden layer:

In [58]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim_list = [100,200,400,800,1600] #size for both hidden layers
dropout = 0.75
for hidden_dim in hidden_dim_list:        
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim) 
            self.hidden2 = nn.Linear(hidden_dim, hidden_dim) #hidden layer 2


            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.relu(self.hidden2(X)) #hidden layer 2
            X = self.dropout(X) #dropout 2
            X = F.softmax(self.output(X), dim=-1)
            return X

    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam,
    callbacks=[EarlyStopping(patience=10)],
    )

    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[hidden_dim] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[hidden_dim] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[hidden_dim] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[hidden_dim]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0173       0.0325        5.0172  1.5052
      2        5.0170       0.0500        5.0165  1.4924
      3        5.0140       0.0301        5.0076  1.5714
      4        5.0031       0.0788        4.9890  1.5027
      5        4.9908       0.1093        4.9624  1.5579
      6        4.9788       0.1215        4.9366  1.5185
      7        4.9692       0.1401        4.9184  1.4710
      8        4.9601       0.1629        4.9013  1.5335
      9        4.9523       0.1904        4.8836  1.5409
     10        4.9459       0.2175        4.8679  1.5013
     11        4.9371       0.2401        4.8478  1.5600
     12        4.9334       0.2666        4.8286  1.6302
     13        4.9212       0.2821        4.8067  1.6952
     14        4.9133       0.2993        4.7890  1.6660
     15        4.9056       0.3215        4.7693  1.6879
     16        4.9029       0.3

    105        4.6772       0.7139        4.3236  1.7308
    106        4.6695       0.7179        4.3224  1.9601
    107        4.6753       0.7169        4.3221  1.6860
    108        4.6763       0.7169        4.3213  1.6749
    109        4.6738       0.7192        4.3196  1.6412
    110        4.6723       0.7202        4.3196  1.6298
    111        4.6664       0.7205        4.3185  1.6467
    112        4.6708       0.7209        4.3181  1.6816
    113        4.6725       0.7222        4.3176  1.6435
    114        4.6575       0.7209        4.3176  1.6305
    115        4.6641       0.7205        4.3168  1.6281
    116        4.6676       0.7219        4.3160  1.6121
    117        4.6675       0.7225        4.3157  1.6401
    118        4.6572       0.7225        4.3155  1.6463
    119        4.6559       0.7209        4.3160  1.6372
    120        4.6652       0.7228        4.3150  1.6174
    121        4.6605       0.7238        4.3136  1.6142
    122        4.6560       0.7

     10        4.7947       0.4666        4.6304  2.1802
     11        4.7689       0.5089        4.5895  2.1720
     12        4.7459       0.5477        4.5494  2.1667
     13        4.7224       0.5636        4.5180  2.1509
     14        4.7036       0.5851        4.4930  2.1955
     15        4.6820       0.5911        4.4761  2.2031
     16        4.6670       0.6020        4.4577  2.1542
     17        4.6522       0.6162        4.4432  2.1363
     18        4.6326       0.6225        4.4326  2.1456
     19        4.6232       0.6301        4.4216  2.1372
     20        4.6095       0.6447        4.4095  2.1892
     21        4.6032       0.6510        4.4026  2.1374
     22        4.5921       0.6533        4.3960  2.1981
     23        4.5835       0.6606        4.3883  2.1991
     24        4.5743       0.6666        4.3805  2.2145
     25        4.5658       0.6738        4.3742  2.1633
     26        4.5563       0.6781        4.3688  2.1547
     27        4.5457       0.6

    118        4.3206       0.8126        4.2198  2.3738
    119        4.3150       0.8132        4.2187  2.4346
    120        4.3137       0.8132        4.2190  2.4416
    121        4.3123       0.8142        4.2181  2.4259
    122        4.3136       0.8126        4.2186  2.4349
    123        4.3180       0.8136        4.2176  2.4401
    124        4.3161       0.8146        4.2174  2.4586
    125        4.3119       0.8142        4.2178  2.4255
    126        4.3087       0.8129        4.2183  2.4132
    127        4.3109       0.8132        4.2182  2.4471
    128        4.3115       0.8152        4.2170  2.3804
    129        4.3088       0.8146        4.2162  2.4103
    130        4.3095       0.8152        4.2162  2.3767
    131        4.3065       0.8139        4.2170  2.4148
    132        4.3048       0.8162        4.2161  2.4213
    133        4.3073       0.8159        4.2158  2.4569
    134        4.2991       0.8149        4.2154  2.4312
    135        4.3053       0.8

     66        4.1296       0.8874        4.1427  4.4876
     67        4.1291       0.8897        4.1420  4.4988
     68        4.1309       0.8907        4.1407  4.4705
     69        4.1260       0.8914        4.1406  4.5457
     70        4.1257       0.8921        4.1404  4.5151
     71        4.1220       0.8914        4.1399  4.5490
     72        4.1228       0.8897        4.1410  4.5413
     73        4.1224       0.8921        4.1406  4.5797
     74        4.1203       0.8921        4.1393  4.5253
     75        4.1178       0.8924        4.1390  4.4683
     76        4.1205       0.8914        4.1402  4.5429
     77        4.1192       0.8930        4.1389  4.5854
     78        4.1185       0.8911        4.1397  4.5606
     79        4.1174       0.8927        4.1385  4.5744
     80        4.1156       0.8947        4.1366  4.5704
     81        4.1143       0.8960        4.1350  4.6004
     82        4.1137       0.8987        4.1328  4.6301
     83        4.1118       0.8

     49        4.0585       0.9252        4.1062  7.9090
     50        4.0574       0.9262        4.1043  7.7395
     51        4.0565       0.9275        4.1028  7.8799
     52        4.0567       0.9288        4.1033  7.9706
     53        4.0553       0.9275        4.1035  7.9312
     54        4.0570       0.9281        4.1033  8.0179
     55        4.0540       0.9288        4.1016  7.9543
     56        4.0542       0.9275        4.1037  7.9625
     57        4.0544       0.9272        4.1043  8.0101
     58        4.0521       0.9281        4.1027  7.9799
     59        4.0519       0.9315        4.0996  7.9870
     60        4.0517       0.9311        4.0996  8.0527
     61        4.0503       0.9318        4.0978  8.0677
     62        4.0497       0.9301        4.0993  8.0485
     63        4.0493       0.9341        4.0977  8.0651
     64        4.0472       0.9325        4.0987  8.1628
     65        4.0474       0.9321        4.0992  8.0849
     66        4.0483       0.9

With no hidden layers?

In [59]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
dropout = 0.75
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=dropout
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.output = nn.Linear(input_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(X)
        X = self.dropout(X) # hidden layer removed
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam,
callbacks=[EarlyStopping(patience=10)],
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[hidden_dim] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[hidden_dim] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[hidden_dim] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[hidden_dim] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[hidden_dim]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)
    

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0170       0.6474        5.0167  1.9125
      2        5.0163       0.7811        5.0160  2.0325
      3        5.0153       0.8086        5.0151  2.0285
      4        5.0140       0.8209        5.0140  1.9413
      5        5.0122       0.8132        5.0127  1.9885
      6        5.0097       0.8017        5.0111  1.9310
      7        5.0061       0.7884        5.0092  2.0014
      8        5.0019       0.7709        5.0070  1.9692
      9        4.9966       0.7623        5.0045  1.9227
     10        4.9915       0.7487        5.0018  1.9665
     11        4.9847       0.7377        4.9988  1.9575
     12        4.9763       0.7295        4.9954  1.9626
     13        4.9668       0.7278        4.9918  2.0229
     14        4.9604       0.7288        4.9879  1.9227
     15        4.9515       0.7288        4.9839  2.0014
     16        4.9407       0.7

    105        4.6747       0.9119        4.5822  1.9365
    106        4.6713       0.9123        4.5792  1.9616
    107        4.6737       0.9119        4.5762  1.9758
    108        4.6696       0.9129        4.5732  1.9211
    109        4.6639       0.9132        4.5702  2.0008
    110        4.6680       0.9119        4.5672  1.9327
    111        4.6708       0.9139        4.5643  2.0023
    112        4.6641       0.9152        4.5614  1.9845
    113        4.6700       0.9159        4.5586  1.9386
    114        4.6723       0.9159        4.5559  2.0182
    115        4.6693       0.9166        4.5530  1.8936
    116        4.6619       0.9172        4.5503  1.9841
    117        4.6667       0.9175        4.5476  1.9503
    118        4.6642       0.9179        4.5448  1.9137
    119        4.6679       0.9182        4.5420  2.0155
    120        4.6600       0.9172        4.5393  1.9281
    121        4.6625       0.9185        4.5367  1.9686
    122        4.6565       0.9

    223        4.6185       0.9232        4.3681  1.9079
    224        4.6150       0.9232        4.3671  1.9789
    225        4.6104       0.9228        4.3660  1.8879
    226        4.6162       0.9225        4.3651  1.9451
    227        4.6165       0.9228        4.3641  1.9179
    228        4.6155       0.9225        4.3631  1.8972
    229        4.6110       0.9219        4.3621  1.9866
    230        4.6103       0.9232        4.3610  1.8940
    231        4.6074       0.9228        4.3600  1.9889
    232        4.6135       0.9225        4.3591  1.9388
    233        4.6092       0.9232        4.3581  1.9166
    234        4.6081       0.9222        4.3573  1.9517
    235        4.6082       0.9222        4.3564  1.9081
    236        4.6099       0.9209        4.3555  1.9884
    237        4.6119       0.9215        4.3547  1.9124
    238        4.6111       0.9215        4.3538  1.9046
    239        4.6120       0.9225        4.3529  1.9184
    240        4.6083       0.9

    345        4.5854       0.9238        4.2866  1.9363
    346        4.5838       0.9238        4.2862  1.9697
    347        4.5884       0.9235        4.2856  1.9723
    348        4.5909       0.9238        4.2852  1.9383
    349        4.5937       0.9235        4.2847  1.9347
    350        4.5853       0.9238        4.2844  1.9175
    351        4.5903       0.9235        4.2840  1.9637
    352        4.5903       0.9238        4.2836  1.9461
    353        4.5904       0.9235        4.2832  1.9950
    354        4.5827       0.9238        4.2828  2.0293
    355        4.5806       0.9248        4.2824  2.0513
    356        4.5840       0.9225        4.2820  2.0689
    357        4.5809       0.9235        4.2815  1.9933
    358        4.5831       0.9235        4.2812  1.9473
    359        4.5836       0.9235        4.2808  1.9251
    360        4.5927       0.9235        4.2805  1.9586
    361        4.5851       0.9235        4.2801  1.9263
    362        4.5802       0.9

    469        4.5709       0.9209        4.2469  1.9316
    470        4.5750       0.9215        4.2466  1.9224
    471        4.5736       0.9222        4.2462  1.9296
    472        4.5755       0.9222        4.2460  1.8886
    473        4.5749       0.9219        4.2456  1.9248
    474        4.5776       0.9209        4.2453  1.9415
    475        4.5702       0.9209        4.2451  2.0847
    476        4.5767       0.9209        4.2448  2.0191
    477        4.5789       0.9212        4.2445  1.9355
    478        4.5743       0.9205        4.2442  1.9756
    479        4.5747       0.9195        4.2441  1.9774
    480        4.5710       0.9199        4.2439  2.0626
    481        4.5778       0.9202        4.2438  1.9492
    482        4.5738       0.9195        4.2436  2.0930
    483        4.5783       0.9205        4.2434  1.9277
    484        4.5724       0.9205        4.2432  1.9513
    485        4.5774       0.9205        4.2429  1.9081
    486        4.5674       0.9

    593        4.5689       0.9189        4.2234  1.9511
    594        4.5673       0.9189        4.2233  1.9108
    595        4.5769       0.9195        4.2231  2.0045
    596        4.5762       0.9189        4.2230  1.9334
    597        4.5674       0.9185        4.2228  2.0055
    598        4.5733       0.9185        4.2225  1.9529
    599        4.5585       0.9179        4.2223  2.0073
    600        4.5653       0.9166        4.2221  1.9913
    601        4.5687       0.9169        4.2220  2.0223
    602        4.5665       0.9175        4.2218  1.9056
    603        4.5732       0.9172        4.2216  1.9439
    604        4.5699       0.9175        4.2216  1.9438
    605        4.5672       0.9166        4.2215  1.9547
    606        4.5715       0.9172        4.2213  1.9337
    607        4.5672       0.9179        4.2212  1.9276
    608        4.5664       0.9172        4.2212  1.9540
    609        4.5686       0.9179        4.2209  1.9410
    610        4.5704       0.9

    717        4.5630       0.9202        4.2073  1.9157
    718        4.5662       0.9199        4.2072  1.9357
    719        4.5771       0.9205        4.2072  1.9320
    720        4.5615       0.9205        4.2071  1.9665
    721        4.5601       0.9205        4.2070  1.9173
    722        4.5703       0.9195        4.2069  1.9356
    723        4.5638       0.9195        4.2068  1.9260
    724        4.5655       0.9195        4.2068  1.9174
    725        4.5653       0.9199        4.2067  1.9570
    726        4.5655       0.9195        4.2066  1.9219
    727        4.5654       0.9192        4.2065  1.9460
    728        4.5654       0.9192        4.2065  1.9477
    729        4.5600       0.9195        4.2064  1.9246
    730        4.5613       0.9192        4.2063  1.9345
    731        4.5616       0.9189        4.2061  1.9013
    732        4.5660       0.9185        4.2060  1.9428
    733        4.5664       0.9189        4.2060  1.9467
    734        4.5646       0.9

    842        4.5644       0.9185        4.1967  1.9283
    843        4.5698       0.9192        4.1965  1.9424
    844        4.5634       0.9185        4.1964  1.9413
    845        4.5605       0.9182        4.1963  1.9491
    846        4.5558       0.9179        4.1962  1.9696
    847        4.5581       0.9182        4.1961  1.9489
    848        4.5578       0.9175        4.1961  1.9454
    849        4.5670       0.9172        4.1961  1.9902
    850        4.5646       0.9182        4.1961  1.9548
    851        4.5562       0.9185        4.1959  1.9737
    852        4.5599       0.9192        4.1959  1.9577
    853        4.5538       0.9199        4.1958  1.9402
    854        4.5672       0.9195        4.1958  1.9305
    855        4.5567       0.9189        4.1957  1.9533
    856        4.5625       0.9195        4.1956  1.9467
    857        4.5611       0.9179        4.1955  1.9337
    858        4.5548       0.9182        4.1954  1.9363
    859        4.5590       0.9

Adjusting for patience:

In [61]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800 #hidden layer size
dropout = 0.75
patience_list = [5, 10, 20, 50]
for patience in patience_list:    #looping for patience
    print(hidden_dim)
    class CLINCModule(nn.Module):
        def __init__(
                self,
                input_dim=vocab_dim,
                hidden_dim=hidden_dim, #setting hidden layer size
                output_dim=output_dim,
                dropout=dropout
        ):
            super(CLINCModule, self).__init__()
            self.dropout = nn.Dropout(dropout)

            self.hidden = nn.Linear(input_dim, hidden_dim)
            self.output = nn.Linear(hidden_dim, output_dim)

        def forward(self, X, **kwargs):
            X = F.relu(self.hidden(X))
            X = self.dropout(X)
            X = F.softmax(self.output(X), dim=-1)
            return X
   
    net = NeuralNetClassifier(
    module=CLINCModule,
    lr=lr,
    criterion=torch.nn.CrossEntropyLoss,
    max_epochs=1000,
    optimizer=torch.optim.Adam, 
    callbacks=[EarlyStopping(patience=patience)], #setting patience 
    )
    
    net.fit(train_x, train_y)
    tlabels = net.predict(train_x)
    tacc[patience] = accuracy_score(tlabels, train_y)
    print('training accuracy')
    print(tacc)
    time0 = time.time()
    labels = net.predict(val_x)
    vacc[patience] = accuracy_score(labels, val_y)
    time1 = time.time()
    vtime[patience] = time1-time0
    print('Val accuracy')
    print(vacc)
    print('pred time')
    print(vtime)
    time2 = time.time()
    olabels = net.predict(val_oos_x)
    oacc[patience] = accuracy_score(olabels, val_oos_y)
    time3 = time.time()
    otime[patience]=time3-time2
    print('OOS Val Accuracy')
    print(oacc)
    print('OOS pred time')
    print(otime)
    

800
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.5245        5.0103  6.6359
      2        4.9139       0.4758        4.7317  6.6599
      3        4.5996       0.6685        4.4816  6.7153
      4        4.4274       0.7589        4.3687  6.4396
      5        4.3246       0.8063        4.2951  6.4540
      6        4.2656       0.8384        4.2550  6.3557
      7        4.2222       0.8609        4.2262  6.3473
      8        4.1939       0.8685        4.2085  6.4061
      9        4.1745       0.8874        4.1913  6.2543
     10        4.1525       0.9043        4.1731  6.3500
     11        4.1296       0.9156        4.1550  6.3983
     12        4.1146       0.9189        4.1475  6.3477
     13        4.1032       0.9225        4.1394  6.3434
     14        4.0940       0.9232        4.1348  6.2272
     15        4.0873       0.9275        4.1310  6.2531
     16        4.0806      

     45        4.0349       0.9434        4.0971  6.3108
     46        4.0348       0.9447        4.0961  6.2760
     47        4.0345       0.9460        4.0955  6.3619
     48        4.0343       0.9454        4.0954  6.3100
     49        4.0346       0.9460        4.0955  6.3138
     50        4.0338       0.9457        4.0954  6.3144
     51        4.0341       0.9447        4.0955  6.4639
     52        4.0335       0.9430        4.0953  6.4119
     53        4.0335       0.9464        4.0941  6.3548
     54        4.0335       0.9454        4.0940  6.3701
     55        4.0336       0.9460        4.0937  6.3463
     56        4.0331       0.9464        4.0937  6.3704
     57        4.0333       0.9450        4.0941  6.3888
     58        4.0330       0.9447        4.0937  6.3430
     59        4.0328       0.9460        4.0941  6.3557
     60        4.0328       0.9454        4.0942  6.3447
     61        4.0329       0.9470        4.0938  6.3282
     62        4.0326       0.9

     80        4.0317       0.9444        4.0926  6.3438
     81        4.0316       0.9450        4.0922  6.3974
     82        4.0315       0.9427        4.0926  6.3608
     83        4.0314       0.9444        4.0922  6.4344
     84        4.0315       0.9440        4.0921  6.4697
     85        4.0315       0.9440        4.0924  6.4249
     86        4.0314       0.9440        4.0920  6.4089
     87        4.0313       0.9467        4.0917  6.3463
     88        4.0310       0.9460        4.0913  6.4175
     89        4.0313       0.9444        4.0920  6.3947
     90        4.0311       0.9437        4.0922  6.4943
     91        4.0312       0.9444        4.0913  6.4630
Stopping since valid_loss has not improved in the last 20 epochs.
training accuracy
{5: 0.9870860927152317, 10: 0.9866887417218543, 20: 0.9874834437086093}
Val accuracy
{5: 0.9066666666666666, 10: 0.909, 20: 0.9066666666666666}
pred time
{5: 0.3598189353942871, 10: 0.38475608825683594, 20: 0.3568098545074463}
OOS V

     98        4.0313       0.9424        4.0942  6.4377
     99        4.0315       0.9454        4.0926  6.4622
    100        4.0313       0.9417        4.0944  6.4508
    101        4.0313       0.9407        4.0935  6.4371
    102        4.0312       0.9434        4.0934  6.3857
    103        4.0312       0.9447        4.0931  6.4095
    104        4.0312       0.9404        4.0944  6.4063
    105        4.0311       0.9417        4.0934  6.6744
    106        4.0311       0.9414        4.0933  6.4796
    107        4.0311       0.9421        4.0931  6.4586
    108        4.0312       0.9427        4.0921  6.4424
    109        4.0311       0.9421        4.0923  6.4655
    110        4.0309       0.9434        4.0928  6.4930
    111        4.0311       0.9401        4.0935  6.4812
    112        4.0311       0.9427        4.0924  6.4479
    113        4.0310       0.9430        4.0920  6.4530
    114        4.0310       0.9424        4.0930  6.4413
    115        4.0310       0.9

Best value for patience of 10 Epochs

No early stopping:

In [63]:
lr = 0.001
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800 
dropout = 0.75

print(hidden_dim)
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim, #setting hidden layer size
            output_dim=output_dim,
            dropout=dropout
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam, #Early Stopping removed
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[patience] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[patience] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[patience] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[patience] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[patience]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)


800
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.5123        5.0102  6.3291
      2        4.9124       0.4752        4.7288  6.2838
      3        4.5918       0.6871        4.4679  6.2390
      4        4.4122       0.7709        4.3588  6.2377
      5        4.3182       0.8123        4.2924  6.2310
      6        4.2627       0.8258        4.2583  6.2400
      7        4.2282       0.8474        4.2322  6.2204
      8        4.2028       0.8579        4.2155  6.1701
      9        4.1816       0.8705        4.2000  6.1621
     10        4.1614       0.8950        4.1813  6.1194
     11        4.1372       0.9076        4.1617  6.1333
     12        4.1199       0.9132        4.1513  6.1468
     13        4.1096       0.9159        4.1455  6.1595
     14        4.1006       0.9172        4.1386  6.2133
     15        4.0922       0.9219        4.1350  6.1740
     16        4.0857      

    120        4.0307       0.9427        4.0917  6.3932
    121        4.0307       0.9424        4.0915  6.4292
    122        4.0305       0.9417        4.0921  6.4065
    123        4.0305       0.9430        4.0922  6.4458
    124        4.0306       0.9437        4.0916  6.3946
    125        4.0305       0.9424        4.0917  6.4726
    126        4.0305       0.9404        4.0925  6.3902
    127        4.0305       0.9424        4.0917  6.4056
    128        4.0306       0.9411        4.0923  6.3937
    129        4.0306       0.9434        4.0919  6.5274
    130        4.0305       0.9430        4.0917  6.4062
    131        4.0305       0.9401        4.0918  6.3949
    132        4.0304       0.9424        4.0911  6.4151
    133        4.0304       0.9421        4.0914  6.4482
    134        4.0304       0.9437        4.0910  6.4010
    135        4.0305       0.9427        4.0915  6.4399
    136        4.0306       0.9411        4.0915  6.4705
    137        4.0305       0.9

    260        4.0301       0.9384        4.0924  6.4510
    261        4.0302       0.9387        4.0916  6.4632
    262        4.0301       0.9397        4.0914  6.4550
    263        4.0301       0.9407        4.0916  6.5228
    264        4.0302       0.9397        4.0923  6.4714
    265        4.0301       0.9404        4.0922  6.4615
    266        4.0301       0.9391        4.0930  6.4507
    267        4.0300       0.9404        4.0922  6.4881
    268        4.0301       0.9394        4.0923  6.4511
    269        4.0301       0.9394        4.0925  6.4433
    270        4.0301       0.9387        4.0933  6.4383
    271        4.0301       0.9377        4.0940  6.5294
    272        4.0300       0.9377        4.0937  6.5002
    273        4.0300       0.9384        4.0930  6.4760
    274        4.0300       0.9361        4.0937  6.5042
    275        4.0301       0.9368        4.0935  6.4472
    276        4.0300       0.9374        4.0935  6.5546
    277        4.0301       0.9

    401        4.0298       0.9384        4.0917  6.8595
    402        4.0299       0.9387        4.0922  6.6630
    403        4.0298       0.9391        4.0918  6.7649
    404        4.0298       0.9381        4.0918  6.6705
    405        4.0298       0.9387        4.0921  6.6471
    406        4.0299       0.9391        4.0917  6.8459
    407        4.0298       0.9384        4.0920  6.6308
    408        4.0299       0.9381        4.0923  6.7126
    409        4.0298       0.9391        4.0919  6.7349
    410        4.0298       0.9377        4.0920  6.6377
    411        4.0298       0.9374        4.0922  6.7885
    412        4.0299       0.9377        4.0923  6.6540
    413        4.0299       0.9381        4.0922  6.6350
    414        4.0298       0.9387        4.0915  6.8223
    415        4.0298       0.9381        4.0915  6.6780
    416        4.0298       0.9384        4.0913  6.7458
    417        4.0298       0.9391        4.0909  6.6639
    418        4.0298       0.9

    543        4.0298       0.9391        4.0922  6.6652
    544        4.0298       0.9387        4.0920  6.6838
    545        4.0298       0.9397        4.0916  6.6728
    546        4.0298       0.9391        4.0916  6.5831
    547        4.0298       0.9397        4.0915  6.9081
    548        4.0298       0.9394        4.0919  6.6190
    549        4.0298       0.9387        4.0922  6.7840
    550        4.0298       0.9397        4.0916  10.2805
    551        4.0298       0.9397        4.0917  286.7433
    552        4.0298       0.9401        4.0908  7.8314
    553        4.0298       0.9391        4.0913  7.2209
    554        4.0298       0.9401        4.0912  7.0530
    555        4.0298       0.9391        4.0912  7.1388
    556        4.0298       0.9387        4.0914  6.8863
    557        4.0298       0.9387        4.0917  6.7048
    558        4.0298       0.9381        4.0920  6.7783
    559        4.0298       0.9404        4.0919  6.6144
    560        4.0298       

    686        4.0298       0.9368        4.0922  12.8274
    687        4.0298       0.9371        4.0924  22.0418
    688        4.0298       0.9361        4.0926  293.2432
    689        4.0298       0.9361        4.0924  11.0351
    690        4.0298       0.9368        4.0926  22.1673
    691        4.0298       0.9371        4.0925  296.3504
    692        4.0298       0.9377        4.0926  7.8375
    693        4.0298       0.9371        4.0922  22.1364
    694        4.0298       0.9374        4.0919  298.4188
    695        4.0298       0.9374        4.0918  6.4436
    696        4.0298       0.9377        4.0918  21.6877
    697        4.0298       0.9368        4.0921  302.1354
    698        4.0298       0.9377        4.0915  6.4095
    699        4.0298       0.9384        4.0918  18.2301
    700        4.0298       0.9377        4.0916  22.3095
    701        4.0298       0.9374        4.0915  6.4356
    702        4.0298       0.9377        4.0914  14.3832
    703       

    827        4.0298       0.9397        4.0907  15.3676
    828        4.0298       0.9397        4.0907  22.9042
    829        4.0298       0.9397        4.0906  290.2831
    830        4.0298       0.9401        4.0908  15.3119
    831        4.0298       0.9397        4.0905  22.9366
    832        4.0298       0.9397        4.0905  291.9086
    833        4.0298       0.9394        4.0904  13.4076
    834        4.0298       0.9394        4.0903  22.9931
    835        4.0298       0.9404        4.0899  293.0152
    836        4.0298       0.9401        4.0904  12.2199
    837        4.0298       0.9391        4.0905  22.9903
    838        4.0298       0.9394        4.0906  293.9956
    839        4.0298       0.9381        4.0906  11.1796
    840        4.0298       0.9387        4.0910  22.8685
    841        4.0298       0.9381        4.0911  296.1385
    842        4.0298       0.9397        4.0911  9.1696
    843        4.0298       0.9394        4.0909  22.9860
    844   

    969        4.0298       0.9394        4.0906  7.9113
    970        4.0298       0.9387        4.0910  7.2865
    971        4.0298       0.9384        4.0908  7.0860
    972        4.0298       0.9377        4.0908  6.9257
    973        4.0298       0.9391        4.0906  6.8468
    974        4.0298       0.9404        4.0902  6.8487
    975        4.0298       0.9404        4.0903  6.9166
    976        4.0298       0.9407        4.0902  6.8827
    977        4.0298       0.9417        4.0900  6.9436
    978        4.0298       0.9401        4.0901  6.9277
    979        4.0298       0.9401        4.0900  6.8692
    980        4.0298       0.9397        4.0905  7.0795
    981        4.0298       0.9397        4.0906  6.9153
    982        4.0298       0.9391        4.0907  7.0382
    983        4.0298       0.9391        4.0908  7.0542
    984        4.0298       0.9401        4.0907  6.9938
    985        4.0298       0.9394        4.0907  7.0106
    986        4.0298       0.9

This is worse. Retraining model using optimal values

In [64]:
lr = 0.001 # best learning rate
tacc={}
vacc = {}
vtime = {}
oacc = {}
otime = {}
hidden_dim = 800 #hidden layer size
dropout = 0.75 #best dropout rate

print(hidden_dim)
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim, 
            output_dim=output_dim,
            dropout=dropout
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim) #one hidden layer only
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X)) #best activation function for hidden layer
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam, #best optimizer.
callbacks=[EarlyStopping(patience=10)], #best patience
)

net.fit(train_x, train_y)
tlabels = net.predict(train_x)
tacc[patience] = accuracy_score(tlabels, train_y)
print('training accuracy')
print(tacc)
time0 = time.time()
labels = net.predict(val_x)
vacc[patience] = accuracy_score(labels, val_y)
time1 = time.time()
vtime[patience] = time1-time0
print('Val accuracy')
print(vacc)
print('pred time')
print(vtime)
time2 = time.time()
olabels = net.predict(val_oos_x)
oacc[patience] = accuracy_score(olabels, val_oos_y)
time3 = time.time()
otime[patience]=time3-time2
print('OOS Val Accuracy')
print(oacc)
print('OOS pred time')
print(otime)



800
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.5401        5.0104  6.2687
      2        4.9119       0.4685        4.7284  6.2613
      3        4.5989       0.6712        4.4784  6.2625
      4        4.4203       0.7672        4.3602  6.2490
      5        4.3205       0.8142        4.2892  6.2845
      6        4.2618       0.8348        4.2541  6.2805
      7        4.2274       0.8454        4.2330  6.3284
      8        4.2029       0.8573        4.2160  6.2509
      9        4.1778       0.8801        4.1933  6.2657
     10        4.1532       0.9020        4.1732  6.1212
     11        4.1310       0.9106        4.1584  6.2055
     12        4.1183       0.9136        4.1505  6.2318
     13        4.1070       0.9166        4.1435  6.2497
     14        4.0961       0.9265        4.1348  6.2474
     15        4.0888       0.9255        4.1310  6.3155
     16        4.0808      

In [66]:
#exporting best model
import pickle
with open('okmlp.pkl', 'wb') as f:
    pickle.dump(net, f)

Timing for best model:

In [67]:
lr = 0.001
hidden_dim = 800 #hidden layer size
dropout = 0.75

print(hidden_dim)
class CLINCModule(nn.Module):
    def __init__(
            self,
            input_dim=vocab_dim,
            hidden_dim=hidden_dim, #setting hidden layer size
            output_dim=output_dim,
            dropout=dropout
    ):
        super(CLINCModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetClassifier(
module=CLINCModule,
lr=lr,
criterion=torch.nn.CrossEntropyLoss,
max_epochs=1000,
optimizer=torch.optim.Adam, #Adam again - momentum has been removed accordingly.
callbacks=[EarlyStopping(patience=10)],
)
timet0 = time.time()
for i in range(10): #taking average over 10 full optimizations
    net.fit(train_x, train_y)
timet1 = time.time()
t_time = (timet1-timet0)/10
print('training time(average of 10 iterations)')
print(t_time)

time0 = time.time()
for i in range(100): #taking average over 100 predictions
    labels = net.predict(val_x)
time1 = time.time()
vtime = (time1-time0)/100
print('validation prediction time (average of 100 iterations)')
print(vtime)


time2 = time.time()
for i in range(100): #taking average over 100 predictions
    olabels = net.predict(val_oos_x)
time3 = time.time()
otime[patience]=(time3-time2)/100
print('OOS prediction time (average of 100 iterations)')
print(otime)

800
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.5652        5.0103  7.3780
      2        4.9108       0.4795        4.7254  6.3650
      3        4.5992       0.6844        4.4755  6.2726
      4        4.4187       0.7679        4.3620  6.3744
      5        4.3210       0.8043        4.2964  6.6267
      6        4.2667       0.8325        4.2560  6.4749
      7        4.2296       0.8497        4.2316  6.4373
      8        4.1983       0.8626        4.2106  6.5754
      9        4.1758       0.8838        4.1920  6.5171
     10        4.1522       0.8983        4.1736  6.4708
     11        4.1334       0.9063        4.1614  6.4083
     12        4.1167       0.9123        4.1510  6.3205
     13        4.1069       0.9129        4.1462  6.3431
     14        4.1000       0.9156        4.1417  6.3934
     15        4.0949       0.9192        4.1380  6.6151
     16        4.0872      

     14        4.0999       0.9156        4.1411  6.2991
     15        4.0943       0.9166        4.1380  6.3786
     16        4.0865       0.9255        4.1306  6.6013
     17        4.0777       0.9291        4.1266  6.6279
     18        4.0717       0.9321        4.1234  6.4790
     19        4.0659       0.9348        4.1206  6.4075
     20        4.0618       0.9354        4.1172  6.3652
     21        4.0571       0.9394        4.1144  6.4991
     22        4.0532       0.9397        4.1117  6.5207
     23        4.0513       0.9427        4.1093  6.3909
     24        4.0488       0.9417        4.1077  6.3628
     25        4.0477       0.9427        4.1068  6.3544
     26        4.0454       0.9407        4.1058  6.4454
     27        4.0444       0.9437        4.1045  6.4014
     28        4.0429       0.9427        4.1041  6.3866
     29        4.0423       0.9434        4.1031  6.5668
     30        4.0411       0.9437        4.1027  6.3973
     31        4.0401       0.9

     25        4.0466       0.9407        4.1066  6.6347
     26        4.0455       0.9417        4.1054  6.7291
     27        4.0439       0.9417        4.1048  6.6612
     28        4.0428       0.9430        4.1045  6.5828
     29        4.0421       0.9447        4.1029  6.5609
     30        4.0412       0.9440        4.1025  6.4501
     31        4.0402       0.9427        4.1017  6.7104
     32        4.0399       0.9421        4.1022  6.7797
     33        4.0387       0.9447        4.1008  6.9062
     34        4.0386       0.9447        4.1006  6.5777
     35        4.0383       0.9467        4.0993  6.5613
     36        4.0375       0.9447        4.0995  6.5703
     37        4.0370       0.9444        4.0996  6.5061
     38        4.0372       0.9427        4.0991  6.4243
     39        4.0366       0.9447        4.0983  6.5010
     40        4.0365       0.9440        4.0979  6.5814
     41        4.0369       0.9447        4.0975  6.5255
     42        4.0360       0.9

     59        4.0328       0.9457        4.0943  6.4243
     60        4.0328       0.9454        4.0938  6.6775
     61        4.0326       0.9460        4.0941  6.5448
     62        4.0325       0.9470        4.0938  6.3740
     63        4.0327       0.9460        4.0935  6.6394
     64        4.0322       0.9467        4.0923  6.6024
     65        4.0323       0.9450        4.0930  6.6287
     66        4.0323       0.9464        4.0927  6.6050
     67        4.0324       0.9450        4.0928  6.6565
     68        4.0320       0.9457        4.0926  6.6289
     69        4.0321       0.9467        4.0927  6.5386
     70        4.0320       0.9450        4.0929  6.6314
     71        4.0319       0.9480        4.0923  6.6163
     72        4.0323       0.9467        4.0929  6.5793
     73        4.0319       0.9444        4.0930  6.5792
Stopping since valid_loss has not improved in the last 10 epochs.
Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    val

      2        4.9137       0.4828        4.7283  6.6732
      3        4.6009       0.6864        4.4767  6.6676
      4        4.4176       0.7685        4.3615  6.3482
      5        4.3219       0.8172        4.2922  6.3929
      6        4.2599       0.8407        4.2485  6.3426
      7        4.2198       0.8546        4.2256  6.2989
      8        4.1964       0.8652        4.2086  6.3067
      9        4.1725       0.8864        4.1915  6.2652
     10        4.1518       0.8983        4.1724  6.3402
     11        4.1307       0.9066        4.1590  6.2250
     12        4.1188       0.9116        4.1507  6.3543
     13        4.1072       0.9136        4.1453  6.3472
     14        4.1005       0.9136        4.1417  6.2645
     15        4.0914       0.9209        4.1352  6.3650
     16        4.0822       0.9288        4.1287  6.3448
     17        4.0762       0.9321        4.1259  6.2964
     18        4.0713       0.9308        4.1229  6.3558
     19        4.0661       0.9

     22        4.0518       0.9411        4.1113  6.6125
     23        4.0503       0.9427        4.1098  6.4819
     24        4.0491       0.9404        4.1082  6.4985
     25        4.0471       0.9427        4.1070  6.4314
     26        4.0452       0.9414        4.1059  6.3382
     27        4.0439       0.9434        4.1047  6.4079
     28        4.0425       0.9430        4.1040  6.5185
     29        4.0427       0.9421        4.1038  6.4130
     30        4.0407       0.9437        4.1031  6.5538
     31        4.0404       0.9424        4.1027  6.4830
     32        4.0398       0.9424        4.1027  6.5192
     33        4.0391       0.9430        4.1018  6.6374
     34        4.0387       0.9407        4.1023  6.5581
     35        4.0378       0.9450        4.1005  6.4688
     36        4.0378       0.9447        4.1004  6.4646
     37        4.0370       0.9447        4.0995  6.4748
     38        4.0367       0.9464        4.0989  6.6906
     39        4.0363       0.9

     62        4.0327       0.9440        4.0945  6.4249
     63        4.0323       0.9450        4.0936  6.4443
     64        4.0323       0.9460        4.0930  6.4440
     65        4.0321       0.9457        4.0934  6.5259
     66        4.0324       0.9450        4.0929  6.5915
     67        4.0322       0.9444        4.0934  6.5167
     68        4.0322       0.9437        4.0942  6.5535
     69        4.0321       0.9444        4.0929  6.6276
     70        4.0320       0.9450        4.0930  6.4476
     71        4.0319       0.9440        4.0928  6.4105
     72        4.0321       0.9447        4.0923  6.4377
     73        4.0319       0.9447        4.0938  6.4220
     74        4.0318       0.9447        4.0929  6.4105
     75        4.0319       0.9457        4.0920  6.4444
     76        4.0317       0.9454        4.0922  6.4296
     77        4.0314       0.9444        4.0925  6.4248
     78        4.0315       0.9437        4.0920  6.4273
     79        4.0316       0.9

     79        4.0317       0.9454        4.0921  6.6292
     80        4.0315       0.9434        4.0922  6.6057
     81        4.0315       0.9450        4.0920  6.4910
     82        4.0317       0.9447        4.0927  6.5474
     83        4.0315       0.9454        4.0910  6.5867
Stopping since valid_loss has not improved in the last 10 epochs.
Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0158       0.4874        5.0102  6.5338
      2        4.9122       0.4864        4.7246  6.6421
      3        4.5979       0.6659        4.4811  6.6101
      4        4.4219       0.7603        4.3626  6.7448
      5        4.3218       0.8142        4.2924  6.5438
      6        4.2583       0.8447        4.2485  6.7564
      7        4.2203       0.8593        4.2238  6.4868
      8        4.1941       0.8692        4.2078  6.2943
      9        4.1748       